In [116]:
import json
import pandas as pd
import requests
import xmltodict
from datetime import datetime

In [49]:
def conform_data(json_data):
    conform_data = {
        "bill": {
            "number": json_data.get("bill", {}).get("number"),
            "updateDate": json_data.get("bill", {}).get("updateDate"),
            "type": json_data.get("bill", {}).get("type"),
            "introducedDate": json_data.get("bill", {}).get("introducedDate"),
            "congress": json_data.get("bill", {}).get("congress"),
            "committees": {
                "item": [item_dict(item) for item in ensure_list(get_item_if_exists(json_data.get("bill", {}).get("committees", {})))]
            },
            "actions": {
                "item": [action_dict(action) for action in ensure_list(get_item_if_exists(json_data.get("bill", {}).get("actions", {})))]
            },
            "sponsors": {
                "item": [sponsor_dict(sponsor) for sponsor in ensure_list(get_item_if_exists(json_data.get("bill", {}).get("sponsors", {})))]
            },
            "cosponsors": {
                "count": json_data.get("bill", {}).get("cosponsors", {}).get("count"),
                "item": [cosponsor_dict(cosponsor) for cosponsor in ensure_list(get_item_if_exists(json_data.get("bill", {}).get("cosponsors", {})))]
            },
            "policyArea": {
                "name": json_data.get("bill", {}).get("policyArea", {}).get("name")
            },
            "subjects": {
                "legislativeSubjects": {
                    "item": [subject_dict(subject) for subject in ensure_list(get_item_if_exists(json_data.get("bill", {}).get("subjects", {}).get("legislativeSubjects", {})))]
                }
            },
            "title": json_data.get("bill", {}).get("title"),
            "latestAction": {
                "actionDate": json_data.get("bill", {}).get("latestAction", {}).get("actionDate"),
                "text": json_data.get("bill", {}).get("latestAction", {}).get("text")
            }
        }
    }

    return conform_data


def item_dict(item):
    if isinstance(item, dict):
        return {
            "name": item.get("name"),
            "chamber": item.get("chamber"),
            "type": item.get("type")
        }
    return {}


def action_dict(action):
    if isinstance(action, dict):
        return {
            "actionDate": action.get("actionDate"),
            "text": action.get("text"),
            "type": action.get("type"),
            "actionCode": action.get("actionCode"),
            "recordedVotes": [vote_dict(vote) for vote in ensure_list(action.get("recordedVotes", {}).get("recordedVote", []))]
        }
    return {}


def sponsor_dict(sponsor):
    if isinstance(sponsor, dict):
        return {
            "bioguideID": sponsor.get("bioguideID"),
            "fulName": sponsor.get("fulName"),
            "firstName": sponsor.get("firstName"),
            "lastName": sponsor.get("lastName")
        }
    return {}


def cosponsor_dict(cosponsor):
    if isinstance(cosponsor, dict):
        return {
            "bioguideID": cosponsor.get("bioguideID")
        }
    return {}


def subject_dict(subject):
    if isinstance(subject, dict):
        return {
            "name": subject.get("name")
        }
    return {}


def vote_dict(vote):
    if isinstance(vote, dict):
        return {
            "rollNumber": vote.get("rollNumber"),
            "url": vote.get("url"),
            "chamber": vote.get("chamber"),
            "congress": vote.get("congress"),
            "date": vote.get("date"),
            "sessionNumber": vote.get("sessionNumber")
        }
    return {}


def ensure_list(item):
    if isinstance(item, list):
        return item
    elif item:
        return [item]
    return []

def get_item_if_exists(input_data):
    if input_data:
        return input_data.get("item", [])
    else:
        return []

In [136]:
def conform_votes(vote_data, bill_number, roll_call_number, chamber):
    conform_data = {
        'vote': {'bill_number': bill_number,
                 'roll_call_number': roll_call_number},
        'legislator': [legislator_dict(item, chamber) for item in vote_data]
    }
    
    return conform_data

def legislator_dict(item, chamber):
        if chamber == 'House':
            if isinstance(item, dict):
                return {
                    'id': item.get("legislator").get("@name-id"),
                    'first_name': None,
                    'last_name': item.get("legislator").get("@unaccented-name"),
                    'party': item.get("legislator").get("@party"),
                    'state': item.get("legislator").get("@state"),
                    'vote' : item.get("vote")
                }
            return {}
        
        if chamber == 'Senate':
            if isinstance(item, dict):
                return {
                    'id': item.get("lis_member_id"),
                    'first_name': item.get("first_name"),
                    'last_name': item.get("last_name"),
                    'party': item.get("party"),
                    'state': item.get("state"),
                    'vote' : item.get("vote_cast")
                }
            return {}

In [137]:
senate_vote = [
    {
        "member_full": "Baldwin (D-WI)",
        "last_name": "Baldwin",
        "first_name": "Tammy",
        "party": "D",
        "state": "WI",
        "vote_cast": "Yea",
        "lis_member_id": "S354"
    },
    {
        "member_full": "Barrasso (R-WY)",
        "last_name": "Barrasso",
        "first_name": "John",
        "party": "R",
        "state": "WY",
        "vote_cast": "Yea",
        "lis_member_id": "S317"
    },
    {
        "member_full": "Bennet (D-CO)",
        "last_name": "Bennet",
        "first_name": "Michael",
        "party": "D",
        "state": "CO",
        "vote_cast": "Yea",
        "lis_member_id": "S330"
    },
    {
        "member_full": "Blackburn (R-TN)",
        "last_name": "Blackburn",
        "first_name": "Marsha",
        "party": "R",
        "state": "TN",
        "vote_cast": "Yea",
        "lis_member_id": "S396"
    },
    {
        "member_full": "Blumenthal (D-CT)",
        "last_name": "Blumenthal",
        "first_name": "Richard",
        "party": "D",
        "state": "CT",
        "vote_cast": "Yea",
        "lis_member_id": "S341"
    },
    {
        "member_full": "Booker (D-NJ)",
        "last_name": "Booker",
        "first_name": "Cory",
        "party": "D",
        "state": "NJ",
        "vote_cast": "Yea",
        "lis_member_id": "S370"
    },
    {
        "member_full": "Boozman (R-AR)",
        "last_name": "Boozman",
        "first_name": "John",
        "party": "R",
        "state": "AR",
        "vote_cast": "Yea",
        "lis_member_id": "S343"
    },
    {
        "member_full": "Braun (R-IN)",
        "last_name": "Braun",
        "first_name": "Mike",
        "party": "R",
        "state": "IN",
        "vote_cast": "Yea",
        "lis_member_id": "S397"
    },
    {
        "member_full": "Britt (R-AL)",
        "last_name": "Britt",
        "first_name": "Katie",
        "party": "R",
        "state": "AL",
        "vote_cast": "Yea",
        "lis_member_id": "S416"
    },
    {
        "member_full": "Brown (D-OH)",
        "last_name": "Brown",
        "first_name": "Sherrod",
        "party": "D",
        "state": "OH",
        "vote_cast": "Yea",
        "lis_member_id": "S307"
    },
    {
        "member_full": "Budd (R-NC)",
        "last_name": "Budd",
        "first_name": "Ted",
        "party": "R",
        "state": "NC",
        "vote_cast": "Yea",
        "lis_member_id": "S417"
    },
    {
        "member_full": "Cantwell (D-WA)",
        "last_name": "Cantwell",
        "first_name": "Maria",
        "party": "D",
        "state": "WA",
        "vote_cast": "Yea",
        "lis_member_id": "S275"
    },
    {
        "member_full": "Capito (R-WV)",
        "last_name": "Capito",
        "first_name": "Shelley",
        "party": "R",
        "state": "WV",
        "vote_cast": "Yea",
        "lis_member_id": "S372"
    },
    {
        "member_full": "Cardin (D-MD)",
        "last_name": "Cardin",
        "first_name": "Ben",
        "party": "D",
        "state": "MD",
        "vote_cast": "Yea",
        "lis_member_id": "S308"
    },
    {
        "member_full": "Carper (D-DE)",
        "last_name": "Carper",
        "first_name": "Thomas",
        "party": "D",
        "state": "DE",
        "vote_cast": "Yea",
        "lis_member_id": "S277"
    },
    {
        "member_full": "Casey (D-PA)",
        "last_name": "Casey",
        "first_name": "Bob",
        "party": "D",
        "state": "PA",
        "vote_cast": "Yea",
        "lis_member_id": "S309"
    },
    {
        "member_full": "Cassidy (R-LA)",
        "last_name": "Cassidy",
        "first_name": "Bill",
        "party": "R",
        "state": "LA",
        "vote_cast": "Yea",
        "lis_member_id": "S373"
    },
    {
        "member_full": "Collins (R-ME)",
        "last_name": "Collins",
        "first_name": "Susan",
        "party": "R",
        "state": "ME",
        "vote_cast": "Yea",
        "lis_member_id": "S252"
    },
    {
        "member_full": "Coons (D-DE)",
        "last_name": "Coons",
        "first_name": "Christopher",
        "party": "D",
        "state": "DE",
        "vote_cast": "Not Voting",
        "lis_member_id": "S337"
    },
    {
        "member_full": "Cornyn (R-TX)",
        "last_name": "Cornyn",
        "first_name": "John",
        "party": "R",
        "state": "TX",
        "vote_cast": "Yea",
        "lis_member_id": "S287"
    },
    {
        "member_full": "Cortez Masto (D-NV)",
        "last_name": "Cortez Masto",
        "first_name": "Catherine",
        "party": "D",
        "state": "NV",
        "vote_cast": "Yea",
        "lis_member_id": "S385"
    },
    {
        "member_full": "Cotton (R-AR)",
        "last_name": "Cotton",
        "first_name": "Tom",
        "party": "R",
        "state": "AR",
        "vote_cast": "Yea",
        "lis_member_id": "S374"
    },
    {
        "member_full": "Cramer (R-ND)",
        "last_name": "Cramer",
        "first_name": "Kevin",
        "party": "R",
        "state": "ND",
        "vote_cast": "Yea",
        "lis_member_id": "S398"
    },
    {
        "member_full": "Crapo (R-ID)",
        "last_name": "Crapo",
        "first_name": "Mike",
        "party": "R",
        "state": "ID",
        "vote_cast": "Yea",
        "lis_member_id": "S266"
    },
    {
        "member_full": "Cruz (R-TX)",
        "last_name": "Cruz",
        "first_name": "Ted",
        "party": "R",
        "state": "TX",
        "vote_cast": "Yea",
        "lis_member_id": "S355"
    },
    {
        "member_full": "Daines (R-MT)",
        "last_name": "Daines",
        "first_name": "Steve",
        "party": "R",
        "state": "MT",
        "vote_cast": "Yea",
        "lis_member_id": "S375"
    },
    {
        "member_full": "Duckworth (D-IL)",
        "last_name": "Duckworth",
        "first_name": "Tammy",
        "party": "D",
        "state": "IL",
        "vote_cast": "Yea",
        "lis_member_id": "S386"
    },
    {
        "member_full": "Durbin (D-IL)",
        "last_name": "Durbin",
        "first_name": "Richard",
        "party": "D",
        "state": "IL",
        "vote_cast": "Yea",
        "lis_member_id": "S253"
    },
    {
        "member_full": "Ernst (R-IA)",
        "last_name": "Ernst",
        "first_name": "Joni",
        "party": "R",
        "state": "IA",
        "vote_cast": "Yea",
        "lis_member_id": "S376"
    },
    {
        "member_full": "Feinstein (D-CA)",
        "last_name": "Feinstein",
        "first_name": "Dianne",
        "party": "D",
        "state": "CA",
        "vote_cast": "Not Voting",
        "lis_member_id": "S221"
    },
    {
        "member_full": "Fetterman (D-PA)",
        "last_name": "Fetterman",
        "first_name": "John",
        "party": "D",
        "state": "PA",
        "vote_cast": "Not Voting",
        "lis_member_id": "S418"
    },
    {
        "member_full": "Fischer (R-NE)",
        "last_name": "Fischer",
        "first_name": "Deb",
        "party": "R",
        "state": "NE",
        "vote_cast": "Yea",
        "lis_member_id": "S357"
    },
    {
        "member_full": "Gillibrand (D-NY)",
        "last_name": "Gillibrand",
        "first_name": "Kirsten",
        "party": "D",
        "state": "NY",
        "vote_cast": "Yea",
        "lis_member_id": "S331"
    },
    {
        "member_full": "Graham (R-SC)",
        "last_name": "Graham",
        "first_name": "Lindsey",
        "party": "R",
        "state": "SC",
        "vote_cast": "Yea",
        "lis_member_id": "S293"
    },
    {
        "member_full": "Grassley (R-IA)",
        "last_name": "Grassley",
        "first_name": "Chuck",
        "party": "R",
        "state": "IA",
        "vote_cast": "Yea",
        "lis_member_id": "S153"
    },
    {
        "member_full": "Hagerty (R-TN)",
        "last_name": "Hagerty",
        "first_name": "Bill",
        "party": "R",
        "state": "TN",
        "vote_cast": "Yea",
        "lis_member_id": "S407"
    },
    {
        "member_full": "Hassan (D-NH)",
        "last_name": "Hassan",
        "first_name": "Maggie",
        "party": "D",
        "state": "NH",
        "vote_cast": "Yea",
        "lis_member_id": "S388"
    },
    {
        "member_full": "Hawley (R-MO)",
        "last_name": "Hawley",
        "first_name": "Josh",
        "party": "R",
        "state": "MO",
        "vote_cast": "Yea",
        "lis_member_id": "S399"
    },
    {
        "member_full": "Heinrich (D-NM)",
        "last_name": "Heinrich",
        "first_name": "Martin",
        "party": "D",
        "state": "NM",
        "vote_cast": "Yea",
        "lis_member_id": "S359"
    },
    {
        "member_full": "Hickenlooper (D-CO)",
        "last_name": "Hickenlooper",
        "first_name": "John",
        "party": "D",
        "state": "CO",
        "vote_cast": "Yea",
        "lis_member_id": "S408"
    },
    {
        "member_full": "Hirono (D-HI)",
        "last_name": "Hirono",
        "first_name": "Mazie",
        "party": "D",
        "state": "HI",
        "vote_cast": "Yea",
        "lis_member_id": "S361"
    },
    {
        "member_full": "Hoeven (R-ND)",
        "last_name": "Hoeven",
        "first_name": "John",
        "party": "R",
        "state": "ND",
        "vote_cast": "Yea",
        "lis_member_id": "S344"
    },
    {
        "member_full": "Hyde-Smith (R-MS)",
        "last_name": "Hyde-Smith",
        "first_name": "Cindy",
        "party": "R",
        "state": "MS",
        "vote_cast": "Yea",
        "lis_member_id": "S395"
    },
    {
        "member_full": "Johnson (R-WI)",
        "last_name": "Johnson",
        "first_name": "Ron",
        "party": "R",
        "state": "WI",
        "vote_cast": "Yea",
        "lis_member_id": "S345"
    },
    {
        "member_full": "Kaine (D-VA)",
        "last_name": "Kaine",
        "first_name": "Timothy",
        "party": "D",
        "state": "VA",
        "vote_cast": "Yea",
        "lis_member_id": "S362"
    },
    {
        "member_full": "Kelly (D-AZ)",
        "last_name": "Kelly",
        "first_name": "Mark",
        "party": "D",
        "state": "AZ",
        "vote_cast": "Yea",
        "lis_member_id": "S406"
    },
    {
        "member_full": "Kennedy (R-LA)",
        "last_name": "Kennedy",
        "first_name": "John",
        "party": "R",
        "state": "LA",
        "vote_cast": "Yea",
        "lis_member_id": "S389"
    },
    {
        "member_full": "King (I-ME)",
        "last_name": "King",
        "first_name": "Angus",
        "party": "I",
        "state": "ME",
        "vote_cast": "Yea",
        "lis_member_id": "S363"
    },
    {
        "member_full": "Klobuchar (D-MN)",
        "last_name": "Klobuchar",
        "first_name": "Amy",
        "party": "D",
        "state": "MN",
        "vote_cast": "Yea",
        "lis_member_id": "S311"
    },
    {
        "member_full": "Lankford (R-OK)",
        "last_name": "Lankford",
        "first_name": "James",
        "party": "R",
        "state": "OK",
        "vote_cast": "Yea",
        "lis_member_id": "S378"
    },
    {
        "member_full": "Lee (R-UT)",
        "last_name": "Lee",
        "first_name": "Mike",
        "party": "R",
        "state": "UT",
        "vote_cast": "Yea",
        "lis_member_id": "S346"
    },
    {
        "member_full": "Lujan (D-NM)",
        "last_name": "Lujan",
        "first_name": "Ben",
        "party": "D",
        "state": "NM",
        "vote_cast": "Yea",
        "lis_member_id": "S409"
    },
    {
        "member_full": "Lummis (R-WY)",
        "last_name": "Lummis",
        "first_name": "Cynthia",
        "party": "R",
        "state": "WY",
        "vote_cast": "Yea",
        "lis_member_id": "S410"
    },
    {
        "member_full": "Manchin (D-WV)",
        "last_name": "Manchin",
        "first_name": "Joseph",
        "party": "D",
        "state": "WV",
        "vote_cast": "Yea",
        "lis_member_id": "S338"
    },
    {
        "member_full": "Markey (D-MA)",
        "last_name": "Markey",
        "first_name": "Edward",
        "party": "D",
        "state": "MA",
        "vote_cast": "Yea",
        "lis_member_id": "S369"
    },
    {
        "member_full": "Marshall (R-KS)",
        "last_name": "Marshall",
        "first_name": "Roger",
        "party": "R",
        "state": "KS",
        "vote_cast": "Yea",
        "lis_member_id": "S411"
    },
    {
        "member_full": "McConnell (R-KY)",
        "last_name": "McConnell",
        "first_name": "Mitch",
        "party": "R",
        "state": "KY",
        "vote_cast": "Not Voting",
        "lis_member_id": "S174"
    },
    {
        "member_full": "Menendez (D-NJ)",
        "last_name": "Menendez",
        "first_name": "Robert",
        "party": "D",
        "state": "NJ",
        "vote_cast": "Yea",
        "lis_member_id": "S306"
    },
    {
        "member_full": "Merkley (D-OR)",
        "last_name": "Merkley",
        "first_name": "Jeff",
        "party": "D",
        "state": "OR",
        "vote_cast": "Yea",
        "lis_member_id": "S322"
    },
    {
        "member_full": "Moran (R-KS)",
        "last_name": "Moran",
        "first_name": "Jerry",
        "party": "R",
        "state": "KS",
        "vote_cast": "Yea",
        "lis_member_id": "S347"
    },
    {
        "member_full": "Mullin (R-OK)",
        "last_name": "Mullin",
        "first_name": "Markwayne",
        "party": "R",
        "state": "OK",
        "vote_cast": "Yea",
        "lis_member_id": "S419"
    },
    {
        "member_full": "Murkowski (R-AK)",
        "last_name": "Murkowski",
        "first_name": "Lisa",
        "party": "R",
        "state": "AK",
        "vote_cast": "Yea",
        "lis_member_id": "S288"
    },
    {
        "member_full": "Murphy (D-CT)",
        "last_name": "Murphy",
        "first_name": "Christopher",
        "party": "D",
        "state": "CT",
        "vote_cast": "Yea",
        "lis_member_id": "S364"
    },
    {
        "member_full": "Murray (D-WA)",
        "last_name": "Murray",
        "first_name": "Patty",
        "party": "D",
        "state": "WA",
        "vote_cast": "Yea",
        "lis_member_id": "S229"
    },
    {
        "member_full": "Ossoff (D-GA)",
        "last_name": "Ossoff",
        "first_name": "Jon",
        "party": "D",
        "state": "GA",
        "vote_cast": "Yea",
        "lis_member_id": "S414"
    },
    {
        "member_full": "Padilla (D-CA)",
        "last_name": "Padilla",
        "first_name": "Alex",
        "party": "D",
        "state": "CA",
        "vote_cast": "Yea",
        "lis_member_id": "S413"
    },
    {
        "member_full": "Paul (R-KY)",
        "last_name": "Paul",
        "first_name": "Rand",
        "party": "R",
        "state": "KY",
        "vote_cast": "Yea",
        "lis_member_id": "S348"
    },
    {
        "member_full": "Peters (D-MI)",
        "last_name": "Peters",
        "first_name": "Gary",
        "party": "D",
        "state": "MI",
        "vote_cast": "Yea",
        "lis_member_id": "S380"
    },
    {
        "member_full": "Reed (D-RI)",
        "last_name": "Reed",
        "first_name": "John",
        "party": "D",
        "state": "RI",
        "vote_cast": "Yea",
        "lis_member_id": "S259"
    },
    {
        "member_full": "Ricketts (R-NE)",
        "last_name": "Ricketts",
        "first_name": "Pete",
        "party": "R",
        "state": "NE",
        "vote_cast": "Yea",
        "lis_member_id": "S423"
    },
    {
        "member_full": "Risch (R-ID)",
        "last_name": "Risch",
        "first_name": "James",
        "party": "R",
        "state": "ID",
        "vote_cast": "Yea",
        "lis_member_id": "S323"
    },
    {
        "member_full": "Romney (R-UT)",
        "last_name": "Romney",
        "first_name": "Mitt",
        "party": "R",
        "state": "UT",
        "vote_cast": "Yea",
        "lis_member_id": "S401"
    },
    {
        "member_full": "Rosen (D-NV)",
        "last_name": "Rosen",
        "first_name": "Jacklyn",
        "party": "D",
        "state": "NV",
        "vote_cast": "Yea",
        "lis_member_id": "S402"
    },
    {
        "member_full": "Rounds (R-SD)",
        "last_name": "Rounds",
        "first_name": "Mike",
        "party": "R",
        "state": "SD",
        "vote_cast": "Yea",
        "lis_member_id": "S381"
    },
    {
        "member_full": "Rubio (R-FL)",
        "last_name": "Rubio",
        "first_name": "Marco",
        "party": "R",
        "state": "FL",
        "vote_cast": "Yea",
        "lis_member_id": "S350"
    },
    {
        "member_full": "Sanders (I-VT)",
        "last_name": "Sanders",
        "first_name": "Bernard",
        "party": "I",
        "state": "VT",
        "vote_cast": "Yea",
        "lis_member_id": "S313"
    },
    {
        "member_full": "Schatz (D-HI)",
        "last_name": "Schatz",
        "first_name": "Brian",
        "party": "D",
        "state": "HI",
        "vote_cast": "Yea",
        "lis_member_id": "S353"
    },
    {
        "member_full": "Schmitt (R-MO)",
        "last_name": "Schmitt",
        "first_name": "Eric",
        "party": "R",
        "state": "MO",
        "vote_cast": "Yea",
        "lis_member_id": "S420"
    },
    {
        "member_full": "Schumer (D-NY)",
        "last_name": "Schumer",
        "first_name": "Charles",
        "party": "D",
        "state": "NY",
        "vote_cast": "Yea",
        "lis_member_id": "S270"
    },
    {
        "member_full": "Scott (R-FL)",
        "last_name": "Scott",
        "first_name": "Rick",
        "party": "R",
        "state": "FL",
        "vote_cast": "Yea",
        "lis_member_id": "S404"
    },
    {
        "member_full": "Scott (R-SC)",
        "last_name": "Scott",
        "first_name": "Tim",
        "party": "R",
        "state": "SC",
        "vote_cast": "Yea",
        "lis_member_id": "S365"
    },
    {
        "member_full": "Shaheen (D-NH)",
        "last_name": "Shaheen",
        "first_name": "Jeanne",
        "party": "D",
        "state": "NH",
        "vote_cast": "Yea",
        "lis_member_id": "S324"
    },
    {
        "member_full": "Sinema (I-AZ)",
        "last_name": "Sinema",
        "first_name": "Kyrsten",
        "party": "I",
        "state": "AZ",
        "vote_cast": "Yea",
        "lis_member_id": "S403"
    },
    {
        "member_full": "Smith (D-MN)",
        "last_name": "Smith",
        "first_name": "Tina",
        "party": "D",
        "state": "MN",
        "vote_cast": "Yea",
        "lis_member_id": "S394"
    },
    {
        "member_full": "Stabenow (D-MI)",
        "last_name": "Stabenow",
        "first_name": "Debbie",
        "party": "D",
        "state": "MI",
        "vote_cast": "Yea",
        "lis_member_id": "S284"
    },
    {
        "member_full": "Sullivan (R-AK)",
        "last_name": "Sullivan",
        "first_name": "Dan",
        "party": "R",
        "state": "AK",
        "vote_cast": "Yea",
        "lis_member_id": "S383"
    },
    {
        "member_full": "Tester (D-MT)",
        "last_name": "Tester",
        "first_name": "Jon",
        "party": "D",
        "state": "MT",
        "vote_cast": "Yea",
        "lis_member_id": "S314"
    },
    {
        "member_full": "Thune (R-SD)",
        "last_name": "Thune",
        "first_name": "John",
        "party": "R",
        "state": "SD",
        "vote_cast": "Yea",
        "lis_member_id": "S303"
    },
    {
        "member_full": "Tillis (R-NC)",
        "last_name": "Tillis",
        "first_name": "Thomas",
        "party": "R",
        "state": "NC",
        "vote_cast": "Yea",
        "lis_member_id": "S384"
    },
    {
        "member_full": "Tuberville (R-AL)",
        "last_name": "Tuberville",
        "first_name": "Tommy",
        "party": "R",
        "state": "AL",
        "vote_cast": "Yea",
        "lis_member_id": "S412"
    },
    {
        "member_full": "Van Hollen (D-MD)",
        "last_name": "Van Hollen",
        "first_name": "Chris",
        "party": "D",
        "state": "MD",
        "vote_cast": "Yea",
        "lis_member_id": "S390"
    },
    {
        "member_full": "Vance (R-OH)",
        "last_name": "Vance",
        "first_name": "J.",
        "party": "R",
        "state": "OH",
        "vote_cast": "Yea",
        "lis_member_id": "S421"
    },
    {
        "member_full": "Warner (D-VA)",
        "last_name": "Warner",
        "first_name": "Mark",
        "party": "D",
        "state": "VA",
        "vote_cast": "Yea",
        "lis_member_id": "S327"
    },
    {
        "member_full": "Warnock (D-GA)",
        "last_name": "Warnock",
        "first_name": "Raphael",
        "party": "D",
        "state": "GA",
        "vote_cast": "Yea",
        "lis_member_id": "S415"
    },
    {
        "member_full": "Warren (D-MA)",
        "last_name": "Warren",
        "first_name": "Elizabeth",
        "party": "D",
        "state": "MA",
        "vote_cast": "Yea",
        "lis_member_id": "S366"
    },
    {
        "member_full": "Welch (D-VT)",
        "last_name": "Welch",
        "first_name": "Peter",
        "party": "D",
        "state": "VT",
        "vote_cast": "Yea",
        "lis_member_id": "S422"
    },
    {
        "member_full": "Whitehouse (D-RI)",
        "last_name": "Whitehouse",
        "first_name": "Sheldon",
        "party": "D",
        "state": "RI",
        "vote_cast": "Yea",
        "lis_member_id": "S316"
    },
    {
        "member_full": "Wicker (R-MS)",
        "last_name": "Wicker",
        "first_name": "Roger",
        "party": "R",
        "state": "MS",
        "vote_cast": "Yea",
        "lis_member_id": "S318"
    },
    {
        "member_full": "Wyden (D-OR)",
        "last_name": "Wyden",
        "first_name": "Ron",
        "party": "D",
        "state": "OR",
        "vote_cast": "Yea",
        "lis_member_id": "S247"
    },
    {
        "member_full": "Young (R-IN)",
        "last_name": "Young",
        "first_name": "Todd",
        "party": "R",
        "state": "IN",
        "vote_cast": "Yea",
        "lis_member_id": "S391"
    }
]

In [138]:
house_vote = [
    {
        "legislator": {
            "@name-id": "A000370",
            "@sort-field": "Adams",
            "@unaccented-name": "Adams",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Adams"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "A000055",
            "@sort-field": "Aderholt",
            "@unaccented-name": "Aderholt",
            "@party": "R",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Aderholt"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000371",
            "@sort-field": "Aguilar",
            "@unaccented-name": "Aguilar",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Aguilar"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000379",
            "@sort-field": "Alford",
            "@unaccented-name": "Alford",
            "@party": "R",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Alford"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000372",
            "@sort-field": "Allen",
            "@unaccented-name": "Allen",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Allen"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000376",
            "@sort-field": "Allred",
            "@unaccented-name": "Allred",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Allred"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000369",
            "@sort-field": "Amodei",
            "@unaccented-name": "Amodei",
            "@party": "R",
            "@state": "NV",
            "@role": "legislator",
            "#text": "Amodei"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000377",
            "@sort-field": "Armstrong",
            "@unaccented-name": "Armstrong",
            "@party": "R",
            "@state": "ND",
            "@role": "legislator",
            "#text": "Armstrong"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000375",
            "@sort-field": "Arrington",
            "@unaccented-name": "Arrington",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Arrington"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "A000148",
            "@sort-field": "Auchincloss",
            "@unaccented-name": "Auchincloss",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Auchincloss"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001291",
            "@sort-field": "Babin",
            "@unaccented-name": "Babin",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Babin"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001298",
            "@sort-field": "Bacon",
            "@unaccented-name": "Bacon",
            "@party": "R",
            "@state": "NE",
            "@role": "legislator",
            "#text": "Bacon"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001307",
            "@sort-field": "Baird",
            "@unaccented-name": "Baird",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Baird"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001306",
            "@sort-field": "Balderson",
            "@unaccented-name": "Balderson",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Balderson"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001318",
            "@sort-field": "Balint",
            "@unaccented-name": "Balint",
            "@party": "D",
            "@state": "VT",
            "@role": "legislator",
            "#text": "Balint"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B001299",
            "@sort-field": "Banks",
            "@unaccented-name": "Banks",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Banks"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001282",
            "@sort-field": "Barr",
            "@unaccented-name": "Barr",
            "@party": "R",
            "@state": "KY",
            "@role": "legislator",
            "#text": "Barr"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001300",
            "@sort-field": "Barragan",
            "@unaccented-name": "Barragan",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Barrag\u00e1n"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B001314",
            "@sort-field": "Bean (FL)",
            "@unaccented-name": "Bean (FL)",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Bean (FL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001281",
            "@sort-field": "Beatty",
            "@unaccented-name": "Beatty",
            "@party": "D",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Beatty"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B000668",
            "@sort-field": "Bentz",
            "@unaccented-name": "Bentz",
            "@party": "R",
            "@state": "OR",
            "@role": "legislator",
            "#text": "Bentz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001287",
            "@sort-field": "Bera",
            "@unaccented-name": "Bera",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Bera"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001301",
            "@sort-field": "Bergman",
            "@unaccented-name": "Bergman",
            "@party": "R",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Bergman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001292",
            "@sort-field": "Beyer",
            "@unaccented-name": "Beyer",
            "@party": "D",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Beyer"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B000740",
            "@sort-field": "Bice",
            "@unaccented-name": "Bice",
            "@party": "R",
            "@state": "OK",
            "@role": "legislator",
            "#text": "Bice"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001302",
            "@sort-field": "Biggs",
            "@unaccented-name": "Biggs",
            "@party": "R",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Biggs"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001257",
            "@sort-field": "Bilirakis",
            "@unaccented-name": "Bilirakis",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Bilirakis"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B000490",
            "@sort-field": "Bishop (GA)",
            "@unaccented-name": "Bishop (GA)",
            "@party": "D",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Bishop (GA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001311",
            "@sort-field": "Bishop (NC)",
            "@unaccented-name": "Bishop (NC)",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Bishop (NC)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B000574",
            "@sort-field": "Blumenauer",
            "@unaccented-name": "Blumenauer",
            "@party": "D",
            "@state": "OR",
            "@role": "legislator",
            "#text": "Blumenauer"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B001303",
            "@sort-field": "Blunt Rochester",
            "@unaccented-name": "Blunt Rochester",
            "@party": "D",
            "@state": "DE",
            "@role": "legislator",
            "#text": "Blunt Rochester"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "B000825",
            "@sort-field": "Boebert",
            "@unaccented-name": "Boebert",
            "@party": "R",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Boebert"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001278",
            "@sort-field": "Bonamici",
            "@unaccented-name": "Bonamici",
            "@party": "D",
            "@state": "OR",
            "@role": "legislator",
            "#text": "Bonamici"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B001295",
            "@sort-field": "Bost",
            "@unaccented-name": "Bost",
            "@party": "R",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Bost"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001223",
            "@sort-field": "Bowman",
            "@unaccented-name": "Bowman",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Bowman"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "B001296",
            "@sort-field": "Boyle (PA)",
            "@unaccented-name": "Boyle (PA)",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Boyle (PA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001317",
            "@sort-field": "Brecheen",
            "@unaccented-name": "Brecheen",
            "@party": "R",
            "@state": "OK",
            "@role": "legislator",
            "#text": "Brecheen"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001313",
            "@sort-field": "Brown",
            "@unaccented-name": "Brown",
            "@party": "D",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Brown"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "B001285",
            "@sort-field": "Brownley",
            "@unaccented-name": "Brownley",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Brownley"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001260",
            "@sort-field": "Buchanan",
            "@unaccented-name": "Buchanan",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Buchanan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001297",
            "@sort-field": "Buck",
            "@unaccented-name": "Buck",
            "@party": "R",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Buck"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001275",
            "@sort-field": "Bucshon",
            "@unaccented-name": "Bucshon",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Bucshon"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001315",
            "@sort-field": "Budzinski",
            "@unaccented-name": "Budzinski",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Budzinski"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001309",
            "@sort-field": "Burchett",
            "@unaccented-name": "Burchett",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Burchett"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001248",
            "@sort-field": "Burgess",
            "@unaccented-name": "Burgess",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Burgess"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001316",
            "@sort-field": "Burlison",
            "@unaccented-name": "Burlison",
            "@party": "R",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Burlison"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "B001224",
            "@sort-field": "Bush",
            "@unaccented-name": "Bush",
            "@party": "D",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Bush"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C000059",
            "@sort-field": "Calvert",
            "@unaccented-name": "Calvert",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Calvert"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001039",
            "@sort-field": "Cammack",
            "@unaccented-name": "Cammack",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Cammack"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001134",
            "@sort-field": "Caraveo",
            "@unaccented-name": "Caraveo",
            "@party": "D",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Caraveo"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001112",
            "@sort-field": "Carbajal",
            "@unaccented-name": "Carbajal",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Carbajal"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001097",
            "@sort-field": "Cardenas",
            "@unaccented-name": "Cardenas",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "C\u00e1rdenas"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001126",
            "@sort-field": "Carey",
            "@unaccented-name": "Carey",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Carey"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001054",
            "@sort-field": "Carl",
            "@unaccented-name": "Carl",
            "@party": "R",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Carl"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001072",
            "@sort-field": "Carson",
            "@unaccented-name": "Carson",
            "@party": "D",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Carson"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001103",
            "@sort-field": "Carter (GA)",
            "@unaccented-name": "Carter (GA)",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Carter (GA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001125",
            "@sort-field": "Carter (LA)",
            "@unaccented-name": "Carter (LA)",
            "@party": "D",
            "@state": "LA",
            "@role": "legislator",
            "#text": "Carter (LA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001051",
            "@sort-field": "Carter (TX)",
            "@unaccented-name": "Carter (TX)",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Carter (TX)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001090",
            "@sort-field": "Cartwright",
            "@unaccented-name": "Cartwright",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Cartwright"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001131",
            "@sort-field": "Casar",
            "@unaccented-name": "Casar",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Casar"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001055",
            "@sort-field": "Case",
            "@unaccented-name": "Case",
            "@party": "D",
            "@state": "HI",
            "@role": "legislator",
            "#text": "Case"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001117",
            "@sort-field": "Casten",
            "@unaccented-name": "Casten",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Casten"
        },
        "vote": "Not Voting"
    },
    {
        "legislator": {
            "@name-id": "C001066",
            "@sort-field": "Castor (FL)",
            "@unaccented-name": "Castor (FL)",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Castor (FL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001091",
            "@sort-field": "Castro (TX)",
            "@unaccented-name": "Castro (TX)",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Castro (TX)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001135",
            "@sort-field": "Chavez-DeRemer",
            "@unaccented-name": "Chavez-DeRemer",
            "@party": "R",
            "@state": "OR",
            "@role": "legislator",
            "#text": "Chavez-DeRemer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001127",
            "@sort-field": "Cherfilus-McCormick",
            "@unaccented-name": "Cherfilus-McCormick",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Cherfilus-McCormick"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001080",
            "@sort-field": "Chu",
            "@unaccented-name": "Chu",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Chu"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001084",
            "@sort-field": "Cicilline",
            "@unaccented-name": "Cicilline",
            "@party": "D",
            "@state": "RI",
            "@role": "legislator",
            "#text": "Cicilline"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "C001133",
            "@sort-field": "Ciscomani",
            "@unaccented-name": "Ciscomani",
            "@party": "R",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Ciscomani"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001101",
            "@sort-field": "Clark (MA)",
            "@unaccented-name": "Clark (MA)",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Clark (MA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001067",
            "@sort-field": "Clarke (NY)",
            "@unaccented-name": "Clarke (NY)",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Clarke (NY)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001061",
            "@sort-field": "Cleaver",
            "@unaccented-name": "Cleaver",
            "@party": "D",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Cleaver"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001118",
            "@sort-field": "Cline",
            "@unaccented-name": "Cline",
            "@party": "R",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Cline"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001115",
            "@sort-field": "Cloud",
            "@unaccented-name": "Cloud",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Cloud"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C000537",
            "@sort-field": "Clyburn",
            "@unaccented-name": "Clyburn",
            "@party": "D",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Clyburn"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001116",
            "@sort-field": "Clyde",
            "@unaccented-name": "Clyde",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Clyde"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001068",
            "@sort-field": "Cohen",
            "@unaccented-name": "Cohen",
            "@party": "D",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Cohen"
        },
        "vote": "Not Voting"
    },
    {
        "legislator": {
            "@name-id": "C001053",
            "@sort-field": "Cole",
            "@unaccented-name": "Cole",
            "@party": "R",
            "@state": "OK",
            "@role": "legislator",
            "#text": "Cole"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001129",
            "@sort-field": "Collins",
            "@unaccented-name": "Collins",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Collins"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001108",
            "@sort-field": "Comer",
            "@unaccented-name": "Comer",
            "@party": "R",
            "@state": "KY",
            "@role": "legislator",
            "#text": "Comer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001078",
            "@sort-field": "Connolly",
            "@unaccented-name": "Connolly",
            "@party": "D",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Connolly"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001110",
            "@sort-field": "Correa",
            "@unaccented-name": "Correa",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Correa"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001059",
            "@sort-field": "Costa",
            "@unaccented-name": "Costa",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Costa"
        },
        "vote": "Not Voting"
    },
    {
        "legislator": {
            "@name-id": "C001069",
            "@sort-field": "Courtney",
            "@unaccented-name": "Courtney",
            "@party": "D",
            "@state": "CT",
            "@role": "legislator",
            "#text": "Courtney"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "C001119",
            "@sort-field": "Craig",
            "@unaccented-name": "Craig",
            "@party": "D",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Craig"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001132",
            "@sort-field": "Crane",
            "@unaccented-name": "Crane",
            "@party": "R",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Crane"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001087",
            "@sort-field": "Crawford",
            "@unaccented-name": "Crawford",
            "@party": "R",
            "@state": "AR",
            "@role": "legislator",
            "#text": "Crawford"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001120",
            "@sort-field": "Crenshaw",
            "@unaccented-name": "Crenshaw",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Crenshaw"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001130",
            "@sort-field": "Crockett",
            "@unaccented-name": "Crockett",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Crockett"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "C001121",
            "@sort-field": "Crow",
            "@unaccented-name": "Crow",
            "@party": "D",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Crow"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001063",
            "@sort-field": "Cuellar",
            "@unaccented-name": "Cuellar",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Cuellar"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "C001114",
            "@sort-field": "Curtis",
            "@unaccented-name": "Curtis",
            "@party": "R",
            "@state": "UT",
            "@role": "legislator",
            "#text": "Curtis"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000632",
            "@sort-field": "D'Esposito",
            "@unaccented-name": "D'Esposito",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "D'Esposito"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000629",
            "@sort-field": "Davids (KS)",
            "@unaccented-name": "Davids (KS)",
            "@party": "D",
            "@state": "KS",
            "@role": "legislator",
            "#text": "Davids (KS)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000626",
            "@sort-field": "Davidson",
            "@unaccented-name": "Davidson",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Davidson"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000096",
            "@sort-field": "Davis (IL)",
            "@unaccented-name": "Davis (IL)",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Davis (IL)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "D000230",
            "@sort-field": "Davis (NC)",
            "@unaccented-name": "Davis (NC)",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Davis (NC)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000594",
            "@sort-field": "De La Cruz",
            "@unaccented-name": "De La Cruz",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "De La Cruz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000631",
            "@sort-field": "Dean (PA)",
            "@unaccented-name": "Dean (PA)",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Dean (PA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000197",
            "@sort-field": "DeGette",
            "@unaccented-name": "DeGette",
            "@party": "D",
            "@state": "CO",
            "@role": "legislator",
            "#text": "DeGette"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "D000216",
            "@sort-field": "DeLauro",
            "@unaccented-name": "DeLauro",
            "@party": "D",
            "@state": "CT",
            "@role": "legislator",
            "#text": "DeLauro"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "D000617",
            "@sort-field": "DelBene",
            "@unaccented-name": "DelBene",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "DelBene"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000530",
            "@sort-field": "Deluzio",
            "@unaccented-name": "Deluzio",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Deluzio"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000623",
            "@sort-field": "DeSaulnier",
            "@unaccented-name": "DeSaulnier",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "DeSaulnier"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "D000616",
            "@sort-field": "DesJarlais",
            "@unaccented-name": "DesJarlais",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "DesJarlais"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000600",
            "@sort-field": "Diaz-Balart",
            "@unaccented-name": "Diaz-Balart",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Diaz-Balart"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000624",
            "@sort-field": "Dingell",
            "@unaccented-name": "Dingell",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Dingell"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000399",
            "@sort-field": "Doggett",
            "@unaccented-name": "Doggett",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Doggett"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "D000032",
            "@sort-field": "Donalds",
            "@unaccented-name": "Donalds",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Donalds"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000633",
            "@sort-field": "Duarte",
            "@unaccented-name": "Duarte",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Duarte"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000615",
            "@sort-field": "Duncan",
            "@unaccented-name": "Duncan",
            "@party": "R",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Duncan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "D000628",
            "@sort-field": "Dunn (FL)",
            "@unaccented-name": "Dunn (FL)",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Dunn (FL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "E000246",
            "@sort-field": "Edwards",
            "@unaccented-name": "Edwards",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Edwards"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "E000071",
            "@sort-field": "Ellzey",
            "@unaccented-name": "Ellzey",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Ellzey"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "E000294",
            "@sort-field": "Emmer",
            "@unaccented-name": "Emmer",
            "@party": "R",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Emmer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "E000299",
            "@sort-field": "Escobar",
            "@unaccented-name": "Escobar",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Escobar"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "E000215",
            "@sort-field": "Eshoo",
            "@unaccented-name": "Eshoo",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Eshoo"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "E000297",
            "@sort-field": "Espaillat",
            "@unaccented-name": "Espaillat",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Espaillat"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "E000298",
            "@sort-field": "Estes",
            "@unaccented-name": "Estes",
            "@party": "R",
            "@state": "KS",
            "@role": "legislator",
            "#text": "Estes"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "E000296",
            "@sort-field": "Evans",
            "@unaccented-name": "Evans",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Evans"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "E000235",
            "@sort-field": "Ezell",
            "@unaccented-name": "Ezell",
            "@party": "R",
            "@state": "MS",
            "@role": "legislator",
            "#text": "Ezell"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000246",
            "@sort-field": "Fallon",
            "@unaccented-name": "Fallon",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Fallon"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000446",
            "@sort-field": "Feenstra",
            "@unaccented-name": "Feenstra",
            "@party": "R",
            "@state": "IA",
            "@role": "legislator",
            "#text": "Feenstra"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000465",
            "@sort-field": "Ferguson",
            "@unaccented-name": "Ferguson",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Ferguson"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000475",
            "@sort-field": "Finstad",
            "@unaccented-name": "Finstad",
            "@party": "R",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Finstad"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000470",
            "@sort-field": "Fischbach",
            "@unaccented-name": "Fischbach",
            "@party": "R",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Fischbach"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000471",
            "@sort-field": "Fitzgerald",
            "@unaccented-name": "Fitzgerald",
            "@party": "R",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Fitzgerald"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000466",
            "@sort-field": "Fitzpatrick",
            "@unaccented-name": "Fitzpatrick",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Fitzpatrick"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000459",
            "@sort-field": "Fleischmann",
            "@unaccented-name": "Fleischmann",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Fleischmann"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000468",
            "@sort-field": "Fletcher",
            "@unaccented-name": "Fletcher",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Fletcher"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "F000474",
            "@sort-field": "Flood",
            "@unaccented-name": "Flood",
            "@party": "R",
            "@state": "NE",
            "@role": "legislator",
            "#text": "Flood"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000454",
            "@sort-field": "Foster",
            "@unaccented-name": "Foster",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Foster"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000477",
            "@sort-field": "Foushee",
            "@unaccented-name": "Foushee",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Foushee"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "F000450",
            "@sort-field": "Foxx",
            "@unaccented-name": "Foxx",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Foxx"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000462",
            "@sort-field": "Frankel, Lois",
            "@unaccented-name": "Frankel, Lois",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Frankel, Lois"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000472",
            "@sort-field": "Franklin, C. Scott",
            "@unaccented-name": "Franklin, C. Scott",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Franklin, C. Scott"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000476",
            "@sort-field": "Frost",
            "@unaccented-name": "Frost",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Frost"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "F000478",
            "@sort-field": "Fry",
            "@unaccented-name": "Fry",
            "@party": "R",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Fry"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "F000469",
            "@sort-field": "Fulcher",
            "@unaccented-name": "Fulcher",
            "@party": "R",
            "@state": "ID",
            "@role": "legislator",
            "#text": "Fulcher"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000578",
            "@sort-field": "Gaetz",
            "@unaccented-name": "Gaetz",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Gaetz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000579",
            "@sort-field": "Gallagher",
            "@unaccented-name": "Gallagher",
            "@party": "R",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Gallagher"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000574",
            "@sort-field": "Gallego",
            "@unaccented-name": "Gallego",
            "@party": "D",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Gallego"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000559",
            "@sort-field": "Garamendi",
            "@unaccented-name": "Garamendi",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Garamendi"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000597",
            "@sort-field": "Garbarino",
            "@unaccented-name": "Garbarino",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Garbarino"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000586",
            "@sort-field": "Garcia (IL)",
            "@unaccented-name": "Garcia (IL)",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Garc\u00eda (IL)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000587",
            "@sort-field": "Garcia (TX)",
            "@unaccented-name": "Garcia (TX)",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Garcia (TX)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000061",
            "@sort-field": "Garcia, Mike",
            "@unaccented-name": "Garcia, Mike",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Garcia, Mike"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000598",
            "@sort-field": "Garcia, Robert",
            "@unaccented-name": "Garcia, Robert",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Garcia, Robert"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000593",
            "@sort-field": "Gimenez",
            "@unaccented-name": "Gimenez",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Gimenez"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000592",
            "@sort-field": "Golden (ME)",
            "@unaccented-name": "Golden (ME)",
            "@party": "D",
            "@state": "ME",
            "@role": "legislator",
            "#text": "Golden (ME)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000599",
            "@sort-field": "Goldman (NY)",
            "@unaccented-name": "Goldman (NY)",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Goldman (NY)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000585",
            "@sort-field": "Gomez",
            "@unaccented-name": "Gomez",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Gomez"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000594",
            "@sort-field": "Gonzales, Tony",
            "@unaccented-name": "Gonzales, Tony",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Gonzales, Tony"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000581",
            "@sort-field": "Gonzalez, Vicente",
            "@unaccented-name": "Gonzalez, Vicente",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Gonzalez, Vicente"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000595",
            "@sort-field": "Good (VA)",
            "@unaccented-name": "Good (VA)",
            "@party": "R",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Good (VA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000589",
            "@sort-field": "Gooden (TX)",
            "@unaccented-name": "Gooden (TX)",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Gooden (TX)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000565",
            "@sort-field": "Gosar",
            "@unaccented-name": "Gosar",
            "@party": "R",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Gosar"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000583",
            "@sort-field": "Gottheimer",
            "@unaccented-name": "Gottheimer",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Gottheimer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000377",
            "@sort-field": "Granger",
            "@unaccented-name": "Granger",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Granger"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000577",
            "@sort-field": "Graves (LA)",
            "@unaccented-name": "Graves (LA)",
            "@party": "R",
            "@state": "LA",
            "@role": "legislator",
            "#text": "Graves (LA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000546",
            "@sort-field": "Graves (MO)",
            "@unaccented-name": "Graves (MO)",
            "@party": "R",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Graves (MO)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000590",
            "@sort-field": "Green (TN)",
            "@unaccented-name": "Green (TN)",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Green (TN)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000553",
            "@sort-field": "Green, Al (TX)",
            "@unaccented-name": "Green, Al (TX)",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Green, Al (TX)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000596",
            "@sort-field": "Greene (GA)",
            "@unaccented-name": "Greene (GA)",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Greene (GA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000568",
            "@sort-field": "Griffith",
            "@unaccented-name": "Griffith",
            "@party": "R",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Griffith"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000551",
            "@sort-field": "Grijalva",
            "@unaccented-name": "Grijalva",
            "@party": "D",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Grijalva"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "G000576",
            "@sort-field": "Grothman",
            "@unaccented-name": "Grothman",
            "@party": "R",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Grothman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000591",
            "@sort-field": "Guest",
            "@unaccented-name": "Guest",
            "@party": "R",
            "@state": "MS",
            "@role": "legislator",
            "#text": "Guest"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "G000558",
            "@sort-field": "Guthrie",
            "@unaccented-name": "Guthrie",
            "@party": "R",
            "@state": "KY",
            "@role": "legislator",
            "#text": "Guthrie"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001096",
            "@sort-field": "Hageman",
            "@unaccented-name": "Hageman",
            "@party": "R",
            "@state": "WY",
            "@role": "legislator",
            "#text": "Hageman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001090",
            "@sort-field": "Harder (CA)",
            "@unaccented-name": "Harder (CA)",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Harder (CA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001052",
            "@sort-field": "Harris",
            "@unaccented-name": "Harris",
            "@party": "R",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Harris"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001086",
            "@sort-field": "Harshbarger",
            "@unaccented-name": "Harshbarger",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Harshbarger"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001081",
            "@sort-field": "Hayes",
            "@unaccented-name": "Hayes",
            "@party": "D",
            "@state": "CT",
            "@role": "legislator",
            "#text": "Hayes"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "H001082",
            "@sort-field": "Hern",
            "@unaccented-name": "Hern",
            "@party": "R",
            "@state": "OK",
            "@role": "legislator",
            "#text": "Hern"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001077",
            "@sort-field": "Higgins (LA)",
            "@unaccented-name": "Higgins (LA)",
            "@party": "R",
            "@state": "LA",
            "@role": "legislator",
            "#text": "Higgins (LA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001038",
            "@sort-field": "Higgins (NY)",
            "@unaccented-name": "Higgins (NY)",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Higgins (NY)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "H001072",
            "@sort-field": "Hill",
            "@unaccented-name": "Hill",
            "@party": "R",
            "@state": "AR",
            "@role": "legislator",
            "#text": "Hill"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001047",
            "@sort-field": "Himes",
            "@unaccented-name": "Himes",
            "@party": "D",
            "@state": "CT",
            "@role": "legislator",
            "#text": "Himes"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "H001091",
            "@sort-field": "Hinson",
            "@unaccented-name": "Hinson",
            "@party": "R",
            "@state": "IA",
            "@role": "legislator",
            "#text": "Hinson"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001066",
            "@sort-field": "Horsford",
            "@unaccented-name": "Horsford",
            "@party": "D",
            "@state": "NV",
            "@role": "legislator",
            "#text": "Horsford"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001093",
            "@sort-field": "Houchin",
            "@unaccented-name": "Houchin",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Houchin"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001085",
            "@sort-field": "Houlahan",
            "@unaccented-name": "Houlahan",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Houlahan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H000874",
            "@sort-field": "Hoyer",
            "@unaccented-name": "Hoyer",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Hoyer"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "H001094",
            "@sort-field": "Hoyle (OR)",
            "@unaccented-name": "Hoyle (OR)",
            "@party": "D",
            "@state": "OR",
            "@role": "legislator",
            "#text": "Hoyle (OR)"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "H001067",
            "@sort-field": "Hudson",
            "@unaccented-name": "Hudson",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Hudson"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001068",
            "@sort-field": "Huffman",
            "@unaccented-name": "Huffman",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Huffman"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "H001058",
            "@sort-field": "Huizenga",
            "@unaccented-name": "Huizenga",
            "@party": "R",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Huizenga"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "H001095",
            "@sort-field": "Hunt",
            "@unaccented-name": "Hunt",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Hunt"
        },
        "vote": "Not Voting"
    },
    {
        "legislator": {
            "@name-id": "I000056",
            "@sort-field": "Issa",
            "@unaccented-name": "Issa",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Issa"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "I000058",
            "@sort-field": "Ivey",
            "@unaccented-name": "Ivey",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Ivey"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000309",
            "@sort-field": "Jackson (IL)",
            "@unaccented-name": "Jackson (IL)",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Jackson (IL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000308",
            "@sort-field": "Jackson (NC)",
            "@unaccented-name": "Jackson (NC)",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Jackson (NC)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000304",
            "@sort-field": "Jackson (TX)",
            "@unaccented-name": "Jackson (TX)",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Jackson (TX)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000032",
            "@sort-field": "Jackson Lee",
            "@unaccented-name": "Jackson Lee",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Jackson Lee"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "J000305",
            "@sort-field": "Jacobs",
            "@unaccented-name": "Jacobs",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Jacobs"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "J000307",
            "@sort-field": "James",
            "@unaccented-name": "James",
            "@party": "R",
            "@state": "MI",
            "@role": "legislator",
            "#text": "James"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000298",
            "@sort-field": "Jayapal",
            "@unaccented-name": "Jayapal",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Jayapal"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "J000294",
            "@sort-field": "Jeffries",
            "@unaccented-name": "Jeffries",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Jeffries"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000288",
            "@sort-field": "Johnson (GA)",
            "@unaccented-name": "Johnson (GA)",
            "@party": "D",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Johnson (GA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "J000299",
            "@sort-field": "Johnson (LA)",
            "@unaccented-name": "Johnson (LA)",
            "@party": "R",
            "@state": "LA",
            "@role": "legislator",
            "#text": "Johnson (LA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000292",
            "@sort-field": "Johnson (OH)",
            "@unaccented-name": "Johnson (OH)",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Johnson (OH)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000301",
            "@sort-field": "Johnson (SD)",
            "@unaccented-name": "Johnson (SD)",
            "@party": "R",
            "@state": "SD",
            "@role": "legislator",
            "#text": "Johnson (SD)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000289",
            "@sort-field": "Jordan",
            "@unaccented-name": "Jordan",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Jordan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000295",
            "@sort-field": "Joyce (OH)",
            "@unaccented-name": "Joyce (OH)",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Joyce (OH)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "J000302",
            "@sort-field": "Joyce (PA)",
            "@unaccented-name": "Joyce (PA)",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Joyce (PA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000400",
            "@sort-field": "Kamlager-Dove",
            "@unaccented-name": "Kamlager-Dove",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Kamlager-Dove"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "K000009",
            "@sort-field": "Kaptur",
            "@unaccented-name": "Kaptur",
            "@party": "D",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Kaptur"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000398",
            "@sort-field": "Kean (NJ)",
            "@unaccented-name": "Kean (NJ)",
            "@party": "R",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Kean (NJ)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000375",
            "@sort-field": "Keating",
            "@unaccented-name": "Keating",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Keating"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000385",
            "@sort-field": "Kelly (IL)",
            "@unaccented-name": "Kelly (IL)",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Kelly (IL)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "K000388",
            "@sort-field": "Kelly (MS)",
            "@unaccented-name": "Kelly (MS)",
            "@party": "R",
            "@state": "MS",
            "@role": "legislator",
            "#text": "Kelly (MS)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000376",
            "@sort-field": "Kelly (PA)",
            "@unaccented-name": "Kelly (PA)",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Kelly (PA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000389",
            "@sort-field": "Khanna",
            "@unaccented-name": "Khanna",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Khanna"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000399",
            "@sort-field": "Kiggans (VA)",
            "@unaccented-name": "Kiggans (VA)",
            "@party": "R",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Kiggans (VA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000380",
            "@sort-field": "Kildee",
            "@unaccented-name": "Kildee",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Kildee"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000401",
            "@sort-field": "Kiley",
            "@unaccented-name": "Kiley",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Kiley"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000381",
            "@sort-field": "Kilmer",
            "@unaccented-name": "Kilmer",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Kilmer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000397",
            "@sort-field": "Kim (CA)",
            "@unaccented-name": "Kim (CA)",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Kim (CA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000394",
            "@sort-field": "Kim (NJ)",
            "@unaccented-name": "Kim (NJ)",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Kim (NJ)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000391",
            "@sort-field": "Krishnamoorthi",
            "@unaccented-name": "Krishnamoorthi",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Krishnamoorthi"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000382",
            "@sort-field": "Kuster",
            "@unaccented-name": "Kuster",
            "@party": "D",
            "@state": "NH",
            "@role": "legislator",
            "#text": "Kuster"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "K000392",
            "@sort-field": "Kustoff",
            "@unaccented-name": "Kustoff",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Kustoff"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000585",
            "@sort-field": "LaHood",
            "@unaccented-name": "LaHood",
            "@party": "R",
            "@state": "IL",
            "@role": "legislator",
            "#text": "LaHood"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000598",
            "@sort-field": "LaLota",
            "@unaccented-name": "LaLota",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "LaLota"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000578",
            "@sort-field": "LaMalfa",
            "@unaccented-name": "LaMalfa",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "LaMalfa"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000564",
            "@sort-field": "Lamborn",
            "@unaccented-name": "Lamborn",
            "@party": "R",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Lamborn"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000601",
            "@sort-field": "Landsman",
            "@unaccented-name": "Landsman",
            "@party": "D",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Landsman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000600",
            "@sort-field": "Langworthy",
            "@unaccented-name": "Langworthy",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Langworthy"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000560",
            "@sort-field": "Larsen (WA)",
            "@unaccented-name": "Larsen (WA)",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Larsen (WA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000557",
            "@sort-field": "Larson (CT)",
            "@unaccented-name": "Larson (CT)",
            "@party": "D",
            "@state": "CT",
            "@role": "legislator",
            "#text": "Larson (CT)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "L000566",
            "@sort-field": "Latta",
            "@unaccented-name": "Latta",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Latta"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000266",
            "@sort-field": "LaTurner",
            "@unaccented-name": "LaTurner",
            "@party": "R",
            "@state": "KS",
            "@role": "legislator",
            "#text": "LaTurner"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000599",
            "@sort-field": "Lawler",
            "@unaccented-name": "Lawler",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Lawler"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000551",
            "@sort-field": "Lee (CA)",
            "@unaccented-name": "Lee (CA)",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Lee (CA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "L000597",
            "@sort-field": "Lee (FL)",
            "@unaccented-name": "Lee (FL)",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Lee (FL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000590",
            "@sort-field": "Lee (NV)",
            "@unaccented-name": "Lee (NV)",
            "@party": "D",
            "@state": "NV",
            "@role": "legislator",
            "#text": "Lee (NV)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000602",
            "@sort-field": "Lee (PA)",
            "@unaccented-name": "Lee (PA)",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Lee (PA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "L000273",
            "@sort-field": "Leger Fernandez",
            "@unaccented-name": "Leger Fernandez",
            "@party": "D",
            "@state": "NM",
            "@role": "legislator",
            "#text": "Leger Fernandez"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "L000589",
            "@sort-field": "Lesko",
            "@unaccented-name": "Lesko",
            "@party": "R",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Lesko"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000595",
            "@sort-field": "Letlow",
            "@unaccented-name": "Letlow",
            "@party": "R",
            "@state": "LA",
            "@role": "legislator",
            "#text": "Letlow"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000593",
            "@sort-field": "Levin",
            "@unaccented-name": "Levin",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Levin"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000582",
            "@sort-field": "Lieu",
            "@unaccented-name": "Lieu",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Lieu"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000397",
            "@sort-field": "Lofgren",
            "@unaccented-name": "Lofgren",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Lofgren"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000583",
            "@sort-field": "Loudermilk",
            "@unaccented-name": "Loudermilk",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Loudermilk"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000491",
            "@sort-field": "Lucas",
            "@unaccented-name": "Lucas",
            "@party": "R",
            "@state": "OK",
            "@role": "legislator",
            "#text": "Lucas"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000569",
            "@sort-field": "Luetkemeyer",
            "@unaccented-name": "Luetkemeyer",
            "@party": "R",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Luetkemeyer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000596",
            "@sort-field": "Luna",
            "@unaccented-name": "Luna",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Luna"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000603",
            "@sort-field": "Luttrell",
            "@unaccented-name": "Luttrell",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Luttrell"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "L000562",
            "@sort-field": "Lynch",
            "@unaccented-name": "Lynch",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Lynch"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M000194",
            "@sort-field": "Mace",
            "@unaccented-name": "Mace",
            "@party": "R",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Mace"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001223",
            "@sort-field": "Magaziner",
            "@unaccented-name": "Magaziner",
            "@party": "D",
            "@state": "RI",
            "@role": "legislator",
            "#text": "Magaziner"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M000317",
            "@sort-field": "Malliotakis",
            "@unaccented-name": "Malliotakis",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Malliotakis"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M000871",
            "@sort-field": "Mann",
            "@unaccented-name": "Mann",
            "@party": "R",
            "@state": "KS",
            "@role": "legislator",
            "#text": "Mann"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001135",
            "@sort-field": "Manning",
            "@unaccented-name": "Manning",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Manning"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001184",
            "@sort-field": "Massie",
            "@unaccented-name": "Massie",
            "@party": "R",
            "@state": "KY",
            "@role": "legislator",
            "#text": "Massie"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001199",
            "@sort-field": "Mast",
            "@unaccented-name": "Mast",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Mast"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001163",
            "@sort-field": "Matsui",
            "@unaccented-name": "Matsui",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Matsui"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "M001208",
            "@sort-field": "McBath",
            "@unaccented-name": "McBath",
            "@party": "D",
            "@state": "GA",
            "@role": "legislator",
            "#text": "McBath"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001165",
            "@sort-field": "McCarthy",
            "@unaccented-name": "McCarthy",
            "@party": "R",
            "@state": "CA",
            "@role": "speaker",
            "#text": "McCarthy"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001157",
            "@sort-field": "McCaul",
            "@unaccented-name": "McCaul",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "McCaul"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001136",
            "@sort-field": "McClain",
            "@unaccented-name": "McClain",
            "@party": "R",
            "@state": "MI",
            "@role": "legislator",
            "#text": "McClain"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001177",
            "@sort-field": "McClintock",
            "@unaccented-name": "McClintock",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "McClintock"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001143",
            "@sort-field": "McCollum",
            "@unaccented-name": "McCollum",
            "@party": "D",
            "@state": "MN",
            "@role": "legislator",
            "#text": "McCollum"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "M001218",
            "@sort-field": "McCormick",
            "@unaccented-name": "McCormick",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "McCormick"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001220",
            "@sort-field": "McGarvey",
            "@unaccented-name": "McGarvey",
            "@party": "D",
            "@state": "KY",
            "@role": "legislator",
            "#text": "McGarvey"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "M000312",
            "@sort-field": "McGovern",
            "@unaccented-name": "McGovern",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "McGovern"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "M001156",
            "@sort-field": "McHenry",
            "@unaccented-name": "McHenry",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "McHenry"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001137",
            "@sort-field": "Meeks",
            "@unaccented-name": "Meeks",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Meeks"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001226",
            "@sort-field": "Menendez",
            "@unaccented-name": "Menendez",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Menendez"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001188",
            "@sort-field": "Meng",
            "@unaccented-name": "Meng",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Meng"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001204",
            "@sort-field": "Meuser",
            "@unaccented-name": "Meuser",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Meuser"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M000687",
            "@sort-field": "Mfume",
            "@unaccented-name": "Mfume",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Mfume"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "M001211",
            "@sort-field": "Miller (IL)",
            "@unaccented-name": "Miller (IL)",
            "@party": "R",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Miller (IL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001222",
            "@sort-field": "Miller (OH)",
            "@unaccented-name": "Miller (OH)",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Miller (OH)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001205",
            "@sort-field": "Miller (WV)",
            "@unaccented-name": "Miller (WV)",
            "@party": "R",
            "@state": "WV",
            "@role": "legislator",
            "#text": "Miller (WV)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001215",
            "@sort-field": "Miller-Meeks",
            "@unaccented-name": "Miller-Meeks",
            "@party": "R",
            "@state": "IA",
            "@role": "legislator",
            "#text": "Miller-Meeks"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001216",
            "@sort-field": "Mills",
            "@unaccented-name": "Mills",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Mills"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001221",
            "@sort-field": "Molinaro",
            "@unaccented-name": "Molinaro",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Molinaro"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001194",
            "@sort-field": "Moolenaar",
            "@unaccented-name": "Moolenaar",
            "@party": "R",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Moolenaar"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001195",
            "@sort-field": "Mooney",
            "@unaccented-name": "Mooney",
            "@party": "R",
            "@state": "WV",
            "@role": "legislator",
            "#text": "Mooney"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001212",
            "@sort-field": "Moore (AL)",
            "@unaccented-name": "Moore (AL)",
            "@party": "R",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Moore (AL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001213",
            "@sort-field": "Moore (UT)",
            "@unaccented-name": "Moore (UT)",
            "@party": "R",
            "@state": "UT",
            "@role": "legislator",
            "#text": "Moore (UT)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001160",
            "@sort-field": "Moore (WI)",
            "@unaccented-name": "Moore (WI)",
            "@party": "D",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Moore (WI)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "M001224",
            "@sort-field": "Moran",
            "@unaccented-name": "Moran",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Moran"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001206",
            "@sort-field": "Morelle",
            "@unaccented-name": "Morelle",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Morelle"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001217",
            "@sort-field": "Moskowitz",
            "@unaccented-name": "Moskowitz",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Moskowitz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001196",
            "@sort-field": "Moulton",
            "@unaccented-name": "Moulton",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Moulton"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001214",
            "@sort-field": "Mrvan",
            "@unaccented-name": "Mrvan",
            "@party": "D",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Mrvan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001225",
            "@sort-field": "Mullin",
            "@unaccented-name": "Mullin",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Mullin"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "M001210",
            "@sort-field": "Murphy",
            "@unaccented-name": "Murphy",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Murphy"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000002",
            "@sort-field": "Nadler",
            "@unaccented-name": "Nadler",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Nadler"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "N000179",
            "@sort-field": "Napolitano",
            "@unaccented-name": "Napolitano",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Napolitano"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "N000015",
            "@sort-field": "Neal",
            "@unaccented-name": "Neal",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Neal"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "N000191",
            "@sort-field": "Neguse",
            "@unaccented-name": "Neguse",
            "@party": "D",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Neguse"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000026",
            "@sort-field": "Nehls",
            "@unaccented-name": "Nehls",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Nehls"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000189",
            "@sort-field": "Newhouse",
            "@unaccented-name": "Newhouse",
            "@party": "R",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Newhouse"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000194",
            "@sort-field": "Nickel",
            "@unaccented-name": "Nickel",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Nickel"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000188",
            "@sort-field": "Norcross",
            "@unaccented-name": "Norcross",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Norcross"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000190",
            "@sort-field": "Norman",
            "@unaccented-name": "Norman",
            "@party": "R",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Norman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "N000193",
            "@sort-field": "Nunn (IA)",
            "@unaccented-name": "Nunn (IA)",
            "@party": "R",
            "@state": "IA",
            "@role": "legislator",
            "#text": "Nunn (IA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "O000019",
            "@sort-field": "Obernolte",
            "@unaccented-name": "Obernolte",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Obernolte"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "O000172",
            "@sort-field": "Ocasio-Cortez",
            "@unaccented-name": "Ocasio-Cortez",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Ocasio-Cortez"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "O000175",
            "@sort-field": "Ogles",
            "@unaccented-name": "Ogles",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Ogles"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "O000173",
            "@sort-field": "Omar",
            "@unaccented-name": "Omar",
            "@party": "D",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Omar"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "O000086",
            "@sort-field": "Owens",
            "@unaccented-name": "Owens",
            "@party": "R",
            "@state": "UT",
            "@role": "legislator",
            "#text": "Owens"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000034",
            "@sort-field": "Pallone",
            "@unaccented-name": "Pallone",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Pallone"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "P000609",
            "@sort-field": "Palmer",
            "@unaccented-name": "Palmer",
            "@party": "R",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Palmer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000613",
            "@sort-field": "Panetta",
            "@unaccented-name": "Panetta",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Panetta"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000614",
            "@sort-field": "Pappas",
            "@unaccented-name": "Pappas",
            "@party": "D",
            "@state": "NH",
            "@role": "legislator",
            "#text": "Pappas"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000096",
            "@sort-field": "Pascrell",
            "@unaccented-name": "Pascrell",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Pascrell"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "P000604",
            "@sort-field": "Payne",
            "@unaccented-name": "Payne",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Payne"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "P000197",
            "@sort-field": "Pelosi",
            "@unaccented-name": "Pelosi",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Pelosi"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000619",
            "@sort-field": "Peltola",
            "@unaccented-name": "Peltola",
            "@party": "D",
            "@state": "AK",
            "@role": "legislator",
            "#text": "Peltola"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000615",
            "@sort-field": "Pence",
            "@unaccented-name": "Pence",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Pence"
        },
        "vote": "Not Voting"
    },
    {
        "legislator": {
            "@name-id": "G000600",
            "@sort-field": "Perez",
            "@unaccented-name": "Perez",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Perez"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000605",
            "@sort-field": "Perry",
            "@unaccented-name": "Perry",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Perry"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000608",
            "@sort-field": "Peters",
            "@unaccented-name": "Peters",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Peters"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000620",
            "@sort-field": "Pettersen",
            "@unaccented-name": "Pettersen",
            "@party": "D",
            "@state": "CO",
            "@role": "legislator",
            "#text": "Pettersen"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000048",
            "@sort-field": "Pfluger",
            "@unaccented-name": "Pfluger",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Pfluger"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000616",
            "@sort-field": "Phillips",
            "@unaccented-name": "Phillips",
            "@party": "D",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Phillips"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000597",
            "@sort-field": "Pingree",
            "@unaccented-name": "Pingree",
            "@party": "D",
            "@state": "ME",
            "@role": "legislator",
            "#text": "Pingree"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "P000607",
            "@sort-field": "Pocan",
            "@unaccented-name": "Pocan",
            "@party": "D",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Pocan"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "P000618",
            "@sort-field": "Porter",
            "@unaccented-name": "Porter",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Porter"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "P000599",
            "@sort-field": "Posey",
            "@unaccented-name": "Posey",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Posey"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "P000617",
            "@sort-field": "Pressley",
            "@unaccented-name": "Pressley",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Pressley"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "Q000023",
            "@sort-field": "Quigley",
            "@unaccented-name": "Quigley",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Quigley"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000617",
            "@sort-field": "Ramirez",
            "@unaccented-name": "Ramirez",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Ramirez"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "R000606",
            "@sort-field": "Raskin",
            "@unaccented-name": "Raskin",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Raskin"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "R000610",
            "@sort-field": "Reschenthaler",
            "@unaccented-name": "Reschenthaler",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Reschenthaler"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "M001159",
            "@sort-field": "Rodgers (WA)",
            "@unaccented-name": "Rodgers (WA)",
            "@party": "R",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Rodgers (WA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000575",
            "@sort-field": "Rogers (AL)",
            "@unaccented-name": "Rogers (AL)",
            "@party": "R",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Rogers (AL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000395",
            "@sort-field": "Rogers (KY)",
            "@unaccented-name": "Rogers (KY)",
            "@party": "R",
            "@state": "KY",
            "@role": "legislator",
            "#text": "Rogers (KY)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000612",
            "@sort-field": "Rose",
            "@unaccented-name": "Rose",
            "@party": "R",
            "@state": "TN",
            "@role": "legislator",
            "#text": "Rose"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000103",
            "@sort-field": "Rosendale",
            "@unaccented-name": "Rosendale",
            "@party": "R",
            "@state": "MT",
            "@role": "legislator",
            "#text": "Rosendale"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000305",
            "@sort-field": "Ross",
            "@unaccented-name": "Ross",
            "@party": "D",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Ross"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "R000603",
            "@sort-field": "Rouzer",
            "@unaccented-name": "Rouzer",
            "@party": "R",
            "@state": "NC",
            "@role": "legislator",
            "#text": "Rouzer"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000614",
            "@sort-field": "Roy",
            "@unaccented-name": "Roy",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Roy"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000599",
            "@sort-field": "Ruiz",
            "@unaccented-name": "Ruiz",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Ruiz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000576",
            "@sort-field": "Ruppersberger",
            "@unaccented-name": "Ruppersberger",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Ruppersberger"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000609",
            "@sort-field": "Rutherford",
            "@unaccented-name": "Rutherford",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Rutherford"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "R000579",
            "@sort-field": "Ryan",
            "@unaccented-name": "Ryan",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Ryan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000168",
            "@sort-field": "Salazar",
            "@unaccented-name": "Salazar",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Salazar"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001226",
            "@sort-field": "Salinas",
            "@unaccented-name": "Salinas",
            "@party": "D",
            "@state": "OR",
            "@role": "legislator",
            "#text": "Salinas"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001156",
            "@sort-field": "Sanchez",
            "@unaccented-name": "Sanchez",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "S\u00e1nchez"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001222",
            "@sort-field": "Santos",
            "@unaccented-name": "Santos",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Santos"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001168",
            "@sort-field": "Sarbanes",
            "@unaccented-name": "Sarbanes",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Sarbanes"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001176",
            "@sort-field": "Scalise",
            "@unaccented-name": "Scalise",
            "@party": "R",
            "@state": "LA",
            "@role": "legislator",
            "#text": "Scalise"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001205",
            "@sort-field": "Scanlon",
            "@unaccented-name": "Scanlon",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Scanlon"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "S001145",
            "@sort-field": "Schakowsky",
            "@unaccented-name": "Schakowsky",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Schakowsky"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001150",
            "@sort-field": "Schiff",
            "@unaccented-name": "Schiff",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Schiff"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001190",
            "@sort-field": "Schneider",
            "@unaccented-name": "Schneider",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Schneider"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001221",
            "@sort-field": "Scholten",
            "@unaccented-name": "Scholten",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Scholten"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001216",
            "@sort-field": "Schrier",
            "@unaccented-name": "Schrier",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Schrier"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001183",
            "@sort-field": "Schweikert",
            "@unaccented-name": "Schweikert",
            "@party": "R",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Schweikert"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000185",
            "@sort-field": "Scott (VA)",
            "@unaccented-name": "Scott (VA)",
            "@party": "D",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Scott (VA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001189",
            "@sort-field": "Scott, Austin",
            "@unaccented-name": "Scott, Austin",
            "@party": "R",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Scott, Austin"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001157",
            "@sort-field": "Scott, David",
            "@unaccented-name": "Scott, David",
            "@party": "D",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Scott, David"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001224",
            "@sort-field": "Self",
            "@unaccented-name": "Self",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Self"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000250",
            "@sort-field": "Sessions",
            "@unaccented-name": "Sessions",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Sessions"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001185",
            "@sort-field": "Sewell",
            "@unaccented-name": "Sewell",
            "@party": "D",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Sewell"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000344",
            "@sort-field": "Sherman",
            "@unaccented-name": "Sherman",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Sherman"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001207",
            "@sort-field": "Sherrill",
            "@unaccented-name": "Sherrill",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Sherrill"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001148",
            "@sort-field": "Simpson",
            "@unaccented-name": "Simpson",
            "@party": "R",
            "@state": "ID",
            "@role": "legislator",
            "#text": "Simpson"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001208",
            "@sort-field": "Slotkin",
            "@unaccented-name": "Slotkin",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Slotkin"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001195",
            "@sort-field": "Smith (MO)",
            "@unaccented-name": "Smith (MO)",
            "@party": "R",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Smith (MO)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001172",
            "@sort-field": "Smith (NE)",
            "@unaccented-name": "Smith (NE)",
            "@party": "R",
            "@state": "NE",
            "@role": "legislator",
            "#text": "Smith (NE)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000522",
            "@sort-field": "Smith (NJ)",
            "@unaccented-name": "Smith (NJ)",
            "@party": "R",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Smith (NJ)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000510",
            "@sort-field": "Smith (WA)",
            "@unaccented-name": "Smith (WA)",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Smith (WA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "S001199",
            "@sort-field": "Smucker",
            "@unaccented-name": "Smucker",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Smucker"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001225",
            "@sort-field": "Sorensen",
            "@unaccented-name": "Sorensen",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Sorensen"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001200",
            "@sort-field": "Soto",
            "@unaccented-name": "Soto",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Soto"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001209",
            "@sort-field": "Spanberger",
            "@unaccented-name": "Spanberger",
            "@party": "D",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Spanberger"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S000929",
            "@sort-field": "Spartz",
            "@unaccented-name": "Spartz",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Spartz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001218",
            "@sort-field": "Stansbury",
            "@unaccented-name": "Stansbury",
            "@party": "D",
            "@state": "NM",
            "@role": "legislator",
            "#text": "Stansbury"
        },
        "vote": "Present"
    },
    {
        "legislator": {
            "@name-id": "S001211",
            "@sort-field": "Stanton",
            "@unaccented-name": "Stanton",
            "@party": "D",
            "@state": "AZ",
            "@role": "legislator",
            "#text": "Stanton"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001212",
            "@sort-field": "Stauber",
            "@unaccented-name": "Stauber",
            "@party": "R",
            "@state": "MN",
            "@role": "legislator",
            "#text": "Stauber"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001135",
            "@sort-field": "Steel",
            "@unaccented-name": "Steel",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Steel"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001196",
            "@sort-field": "Stefanik",
            "@unaccented-name": "Stefanik",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Stefanik"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001213",
            "@sort-field": "Steil",
            "@unaccented-name": "Steil",
            "@party": "R",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Steil"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001214",
            "@sort-field": "Steube",
            "@unaccented-name": "Steube",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Steube"
        },
        "vote": "Not Voting"
    },
    {
        "legislator": {
            "@name-id": "S001215",
            "@sort-field": "Stevens",
            "@unaccented-name": "Stevens",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Stevens"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001192",
            "@sort-field": "Stewart",
            "@unaccented-name": "Stewart",
            "@party": "R",
            "@state": "UT",
            "@role": "legislator",
            "#text": "Stewart"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001159",
            "@sort-field": "Strickland",
            "@unaccented-name": "Strickland",
            "@party": "D",
            "@state": "WA",
            "@role": "legislator",
            "#text": "Strickland"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001220",
            "@sort-field": "Strong",
            "@unaccented-name": "Strong",
            "@party": "R",
            "@state": "AL",
            "@role": "legislator",
            "#text": "Strong"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001193",
            "@sort-field": "Swalwell",
            "@unaccented-name": "Swalwell",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Swalwell"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "S001223",
            "@sort-field": "Sykes",
            "@unaccented-name": "Sykes",
            "@party": "D",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Sykes"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000472",
            "@sort-field": "Takano",
            "@unaccented-name": "Takano",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Takano"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "T000478",
            "@sort-field": "Tenney",
            "@unaccented-name": "Tenney",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Tenney"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000488",
            "@sort-field": "Thanedar",
            "@unaccented-name": "Thanedar",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Thanedar"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "T000460",
            "@sort-field": "Thompson (CA)",
            "@unaccented-name": "Thompson (CA)",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Thompson (CA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000193",
            "@sort-field": "Thompson (MS)",
            "@unaccented-name": "Thompson (MS)",
            "@party": "D",
            "@state": "MS",
            "@role": "legislator",
            "#text": "Thompson (MS)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000467",
            "@sort-field": "Thompson (PA)",
            "@unaccented-name": "Thompson (PA)",
            "@party": "R",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Thompson (PA)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000165",
            "@sort-field": "Tiffany",
            "@unaccented-name": "Tiffany",
            "@party": "R",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Tiffany"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000480",
            "@sort-field": "Timmons",
            "@unaccented-name": "Timmons",
            "@party": "R",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Timmons"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000468",
            "@sort-field": "Titus",
            "@unaccented-name": "Titus",
            "@party": "D",
            "@state": "NV",
            "@role": "legislator",
            "#text": "Titus"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000481",
            "@sort-field": "Tlaib",
            "@unaccented-name": "Tlaib",
            "@party": "D",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Tlaib"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "T000487",
            "@sort-field": "Tokuda",
            "@unaccented-name": "Tokuda",
            "@party": "D",
            "@state": "HI",
            "@role": "legislator",
            "#text": "Tokuda"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "T000469",
            "@sort-field": "Tonko",
            "@unaccented-name": "Tonko",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Tonko"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "T000474",
            "@sort-field": "Torres (CA)",
            "@unaccented-name": "Torres (CA)",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Torres (CA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "T000486",
            "@sort-field": "Torres (NY)",
            "@unaccented-name": "Torres (NY)",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Torres (NY)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000482",
            "@sort-field": "Trahan",
            "@unaccented-name": "Trahan",
            "@party": "D",
            "@state": "MA",
            "@role": "legislator",
            "#text": "Trahan"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000483",
            "@sort-field": "Trone",
            "@unaccented-name": "Trone",
            "@party": "D",
            "@state": "MD",
            "@role": "legislator",
            "#text": "Trone"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "T000463",
            "@sort-field": "Turner",
            "@unaccented-name": "Turner",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Turner"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "U000040",
            "@sort-field": "Underwood",
            "@unaccented-name": "Underwood",
            "@party": "D",
            "@state": "IL",
            "@role": "legislator",
            "#text": "Underwood"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000129",
            "@sort-field": "Valadao",
            "@unaccented-name": "Valadao",
            "@party": "R",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Valadao"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000133",
            "@sort-field": "Van Drew",
            "@unaccented-name": "Van Drew",
            "@party": "R",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Van Drew"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000134",
            "@sort-field": "Van Duyne",
            "@unaccented-name": "Van Duyne",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Van Duyne"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000135",
            "@sort-field": "Van Orden",
            "@unaccented-name": "Van Orden",
            "@party": "R",
            "@state": "WI",
            "@role": "legislator",
            "#text": "Van Orden"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000130",
            "@sort-field": "Vargas",
            "@unaccented-name": "Vargas",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Vargas"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "V000136",
            "@sort-field": "Vasquez",
            "@unaccented-name": "Vasquez",
            "@party": "D",
            "@state": "NM",
            "@role": "legislator",
            "#text": "Vasquez"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000131",
            "@sort-field": "Veasey",
            "@unaccented-name": "Veasey",
            "@party": "D",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Veasey"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "V000081",
            "@sort-field": "Velazquez",
            "@unaccented-name": "Velazquez",
            "@party": "D",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Vel\u00e1zquez"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "W000812",
            "@sort-field": "Wagner",
            "@unaccented-name": "Wagner",
            "@party": "R",
            "@state": "MO",
            "@role": "legislator",
            "#text": "Wagner"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000798",
            "@sort-field": "Walberg",
            "@unaccented-name": "Walberg",
            "@party": "R",
            "@state": "MI",
            "@role": "legislator",
            "#text": "Walberg"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000823",
            "@sort-field": "Waltz",
            "@unaccented-name": "Waltz",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Waltz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000797",
            "@sort-field": "Wasserman Schultz",
            "@unaccented-name": "Wasserman Schultz",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Wasserman Schultz"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000187",
            "@sort-field": "Waters",
            "@unaccented-name": "Waters",
            "@party": "D",
            "@state": "CA",
            "@role": "legislator",
            "#text": "Waters"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "W000822",
            "@sort-field": "Watson Coleman",
            "@unaccented-name": "Watson Coleman",
            "@party": "D",
            "@state": "NJ",
            "@role": "legislator",
            "#text": "Watson Coleman"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "W000814",
            "@sort-field": "Weber (TX)",
            "@unaccented-name": "Weber (TX)",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Weber (TX)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000806",
            "@sort-field": "Webster (FL)",
            "@unaccented-name": "Webster (FL)",
            "@party": "R",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Webster (FL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000815",
            "@sort-field": "Wenstrup",
            "@unaccented-name": "Wenstrup",
            "@party": "R",
            "@state": "OH",
            "@role": "legislator",
            "#text": "Wenstrup"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000821",
            "@sort-field": "Westerman",
            "@unaccented-name": "Westerman",
            "@party": "R",
            "@state": "AR",
            "@role": "legislator",
            "#text": "Westerman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000825",
            "@sort-field": "Wexton",
            "@unaccented-name": "Wexton",
            "@party": "D",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Wexton"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000826",
            "@sort-field": "Wild",
            "@unaccented-name": "Wild",
            "@party": "D",
            "@state": "PA",
            "@role": "legislator",
            "#text": "Wild"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000788",
            "@sort-field": "Williams (GA)",
            "@unaccented-name": "Williams (GA)",
            "@party": "D",
            "@state": "GA",
            "@role": "legislator",
            "#text": "Williams (GA)"
        },
        "vote": "Nay"
    },
    {
        "legislator": {
            "@name-id": "W000828",
            "@sort-field": "Williams (NY)",
            "@unaccented-name": "Williams (NY)",
            "@party": "R",
            "@state": "NY",
            "@role": "legislator",
            "#text": "Williams (NY)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000816",
            "@sort-field": "Williams (TX)",
            "@unaccented-name": "Williams (TX)",
            "@party": "R",
            "@state": "TX",
            "@role": "legislator",
            "#text": "Williams (TX)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000808",
            "@sort-field": "Wilson (FL)",
            "@unaccented-name": "Wilson (FL)",
            "@party": "D",
            "@state": "FL",
            "@role": "legislator",
            "#text": "Wilson (FL)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000795",
            "@sort-field": "Wilson (SC)",
            "@unaccented-name": "Wilson (SC)",
            "@party": "R",
            "@state": "SC",
            "@role": "legislator",
            "#text": "Wilson (SC)"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000804",
            "@sort-field": "Wittman",
            "@unaccented-name": "Wittman",
            "@party": "R",
            "@state": "VA",
            "@role": "legislator",
            "#text": "Wittman"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "W000809",
            "@sort-field": "Womack",
            "@unaccented-name": "Womack",
            "@party": "R",
            "@state": "AR",
            "@role": "legislator",
            "#text": "Womack"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "Y000067",
            "@sort-field": "Yakym",
            "@unaccented-name": "Yakym",
            "@party": "R",
            "@state": "IN",
            "@role": "legislator",
            "#text": "Yakym"
        },
        "vote": "Yea"
    },
    {
        "legislator": {
            "@name-id": "Z000018",
            "@sort-field": "Zinke",
            "@unaccented-name": "Zinke",
            "@party": "R",
            "@state": "MT",
            "@role": "legislator",
            "#text": "Zinke"
        },
        "vote": "Yea"
    }
]

In [139]:
conform_votes(vote_data=senate_vote, bill_number=1, roll_call_number=1, chamber='Senate')

{'vote': {'bill_number': 1, 'roll_call_number': 1},
 'legislator': [{'id': 'S354',
   'first_name': 'Tammy',
   'last_name': 'Baldwin',
   'party': 'D',
   'state': 'WI',
   'vote': 'Yea'},
  {'id': 'S317',
   'first_name': 'John',
   'last_name': 'Barrasso',
   'party': 'R',
   'state': 'WY',
   'vote': 'Yea'},
  {'id': 'S330',
   'first_name': 'Michael',
   'last_name': 'Bennet',
   'party': 'D',
   'state': 'CO',
   'vote': 'Yea'},
  {'id': 'S396',
   'first_name': 'Marsha',
   'last_name': 'Blackburn',
   'party': 'R',
   'state': 'TN',
   'vote': 'Yea'},
  {'id': 'S341',
   'first_name': 'Richard',
   'last_name': 'Blumenthal',
   'party': 'D',
   'state': 'CT',
   'vote': 'Yea'},
  {'id': 'S370',
   'first_name': 'Cory',
   'last_name': 'Booker',
   'party': 'D',
   'state': 'NJ',
   'vote': 'Yea'},
  {'id': 'S343',
   'first_name': 'John',
   'last_name': 'Boozman',
   'party': 'R',
   'state': 'AR',
   'vote': 'Yea'},
  {'id': 'S397',
   'first_name': 'Mike',
   'last_name': 'B

In [140]:
def conform_senate_votes(vote_data, bill_number, roll_call_number):
    conform_data = {
        'vote': {'bill_number': bill_number,
                 'roll_call_number': roll_call_number},
        'legislator': [house_legislator_dict(item.get("legislator")) for item in vote_data]
    }
    
    return conform_data

def senate_legislator_dict(legislator):
        if isinstance(legislator, dict):
            return {
                'id': legislator.get("@name-id"),
                'first_name': None,
                'last_name': legislator.get("@unaccented-name"),
                'party': legislator.get("@party"),
                'state': legislator.get("@state")
            }
        return {}
        

In [141]:
input_json = r"""
{
    "billStatus": {
        "version": "3.0.0",
        "bill": {
            "number": "21",
            "updateDate": "2024-03-18T21:10:34Z",
            "updateDateIncludingText": "2024-03-18T21:10:34Z",
            "originChamber": "House",
            "originChamberCode": "H",
            "type": "HCONRES",
            "introducedDate": "2023-03-01",
            "congress": "118",
            "cdata": null,
            "committees": {
                "item": {
                    "systemCode": "hsfa00",
                    "name": "Foreign Affairs Committee",
                    "chamber": "House",
                    "type": "Standing",
                    "activities": {
                        "item": {
                            "name": "Referred to",
                            "date": "2023-03-01T14:09:45Z"
                        }
                    }
                }
            },
            "actions": {
                "item": [
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "18:00:44",
                        "text": "Motion to reconsider laid on the table Agreed to without objection.",
                        "type": "Floor",
                        "actionCode": "H38310",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "18:00:41",
                        "text": "On agreeing to the resolution Failed by the Yeas and Nays: 103 - 321 (Roll no. 136).",
                        "type": "Floor",
                        "actionCode": "H37100",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        },
                        "recordedVotes": {
                            "recordedVote": {
                                "rollNumber": "136",
                                "url": "https://clerk.house.gov/evs/2023/roll136.xml",
                                "chamber": "House",
                                "congress": "118",
                                "date": "2023-03-08T23:00:41Z",
                                "sessionNumber": "1"
                            }
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "18:00:41",
                        "text": "Failed of passage/not agreed to in House On agreeing to the resolution Failed by the Yeas and Nays: 103 - 321 (Roll no. 136).",
                        "type": "Floor",
                        "actionCode": "9000",
                        "sourceSystem": {
                            "code": "9",
                            "name": "Library of Congress"
                        },
                        "recordedVotes": {
                            "recordedVote": {
                                "rollNumber": "136",
                                "url": "https://clerk.house.gov/evs/2023/roll136.xml",
                                "chamber": "House",
                                "congress": "118",
                                "date": "2023-03-08T23:00:41Z",
                                "sessionNumber": "1"
                            }
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "17:32:53",
                        "text": "Considered as unfinished business.",
                        "type": "Floor",
                        "actionCode": "H30000",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "15:12:24",
                        "text": "POSTPONED PROCEEDINGS - At the conclusion of debate on H. Con. Res. 21, the Chair put the question on agreeing to the resolution and by voice vote announced that the noes had prevailed. Mr. Gaetz demanded the yeas and nays and the Chair postponed further proceedings on agreeing to the resolution until a time to be announced.",
                        "type": "Floor",
                        "actionCode": "H8D000",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "15:12:14",
                        "text": "The previous question was ordered pursuant to a previous order of the House.",
                        "type": "Floor",
                        "actionCode": "H35000",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "14:11:23",
                        "text": "DEBATE - Pursuant to a previous order of the House, the House proceeded with one hour of debate on H. Con. Res. 21.",
                        "type": "Floor",
                        "actionCode": "H8D000",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "14:10:45",
                        "text": "Considered pursuant to a previous order.",
                        "type": "Floor",
                        "actionCode": "H30000",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "14:09:57",
                        "text": "Consideration initiated pursuant to a previous order.",
                        "type": "Floor",
                        "actionCode": "H30800",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-08",
                        "actionTime": "12:20:01",
                        "text": "ORDER OF PROCEDURE - Mr. Massie asked for unanimous consent that it be in order at any time to consider H.Con.Res. 21 in the House if called up by the chair of the Committee on Foreign Affairs or his designee; that the concurrent resolution be considered as read; that the previous question be considered as ordered on the concurrent resolution to adoption without intervening motion except for one hour of debate equally divided among and controlled by Representative McCaul, Representative Meeks, and Representative Gaetz or their respective designees; and that the provisions of section 7 of the War Powers, Resolution, 50 U.S.C 1546, shall not apply to H.Con.Res. 20. Agreed to without objection.",
                        "type": "Floor",
                        "actionCode": "H8D000",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        }
                    },
                    {
                        "actionDate": "2023-03-01",
                        "text": "Referred to the House Committee on Foreign Affairs.",
                        "type": "IntroReferral",
                        "actionCode": "H11100",
                        "sourceSystem": {
                            "code": "2",
                            "name": "House floor actions"
                        },
                        "committees": {
                            "item": {
                                "systemCode": "hsfa00",
                                "name": "Foreign Affairs Committee"
                            }
                        }
                    },
                    {
                        "actionDate": "2023-03-01",
                        "text": "Introduced in House",
                        "type": "IntroReferral",
                        "actionCode": "Intro-H",
                        "sourceSystem": {
                            "code": "9",
                            "name": "Library of Congress"
                        }
                    },
                    {
                        "actionDate": "2023-03-01",
                        "text": "Introduced in House",
                        "type": "IntroReferral",
                        "actionCode": "1000",
                        "sourceSystem": {
                            "code": "9",
                            "name": "Library of Congress"
                        }
                    }
                ]
            },
            "sponsors": {
                "item": {
                    "bioguideId": "G000578",
                    "fullName": "Rep. Gaetz, Matt [R-FL-1]",
                    "firstName": "Matt",
                    "lastName": "Gaetz",
                    "party": "R",
                    "state": "FL",
                    "district": "1",
                    "isByRequest": "N"
                }
            },
            "cosponsors": {
                "item": [
                    {
                        "bioguideId": "B001302",
                        "fullName": "Rep. Biggs, Andy [R-AZ-5]",
                        "firstName": "Andy",
                        "lastName": "Biggs",
                        "party": "R",
                        "state": "AZ",
                        "district": "5",
                        "sponsorshipDate": "2023-03-03",
                        "isOriginalCosponsor": "False"
                    },
                    {
                        "bioguideId": "G000565",
                        "fullName": "Rep. Gosar, Paul A. [R-AZ-9]",
                        "firstName": "Paul",
                        "lastName": "Gosar",
                        "party": "R",
                        "state": "AZ",
                        "middleName": "A.",
                        "district": "9",
                        "sponsorshipDate": "2023-03-07",
                        "isOriginalCosponsor": "False"
                    },
                    {
                        "bioguideId": "G000596",
                        "fullName": "Rep. Greene, Marjorie Taylor [R-GA-14]",
                        "firstName": "Marjorie",
                        "lastName": "Greene",
                        "party": "R",
                        "state": "GA",
                        "middleName": "Taylor",
                        "district": "14",
                        "sponsorshipDate": "2023-03-07",
                        "isOriginalCosponsor": "False"
                    },
                    {
                        "bioguideId": "R000103",
                        "fullName": "Rep. Rosendale Sr., Matthew M. [R-MT-2]",
                        "firstName": "Matthew",
                        "lastName": "Rosendale",
                        "party": "R",
                        "state": "MT",
                        "middleName": "M.",
                        "district": "2",
                        "sponsorshipDate": "2023-03-07",
                        "isOriginalCosponsor": "False"
                    }
                ]
            },
            "summaries": {
                "summary": {
                    "versionCode": "00",
                    "actionDate": "2023-03-01",
                    "actionDesc": "Introduced in House",
                    "updateDate": "2023-03-06T18:28:20Z",
                    "text": "<p>This concurrent resolution directs the President to remove the U.S. Armed Forces from Syria within 180 days of the adoption of this concurrent resolution.</p>"
                }
            },
            "title": "Directing the President, pursuant to section 5(c) of the War Powers Resolution, to remove the United States Armed Forces from Syria.",
            "titles": {
                "item": [
                    {
                        "titleType": "Display Title",
                        "title": "Directing the President, pursuant to section 5(c) of the War Powers Resolution, to remove the United States Armed Forces from Syria."
                    },
                    {
                        "titleType": "Official Title as Introduced",
                        "title": "Directing the President, pursuant to section 5(c) of the War Powers Resolution, to remove the United States Armed Forces from Syria.",
                        "billTextVersionName": "Introduced in House",
                        "billTextVersionCode": "IH"
                    }
                ]
            },
            "textVersions": {
                "item": {
                    "type": "Introduced in House",
                    "date": "2023-03-01T05:00:00Z",
                    "formats": {
                        "item": {
                            "url": "https://www.govinfo.gov/content/pkg/BILLS-118hconres21ih/xml/BILLS-118hconres21ih.xml"
                        }
                    }
                }
            },
            "latestAction": {
                "actionDate": "2023-03-08",
                "text": "Motion to reconsider laid on the table Agreed to without objection.",
                "actionTime": "18:00:44"
            }
        },
        "dublinCore": {
            "xmlnsdc": "http://purl.org/dc/elements/1.1/",
            "dcformat": "text/xml",
            "dclanguage": "EN",
            "dcrights": "Pursuant to Title 17 Section 105 of the United States Code, this file is not subject to copyright protection and is in the public domain.",
            "dccontributor": "Congressional Research Service, Library of Congress",
            "dcdescription": "This file contains bill summaries and statuses for federal legislation. A bill summary describes the most significant provisions of a piece of legislation and details the effects the legislative text may have on current law and federal programs. Bill summaries are authored by the Congressional Research Service (CRS) of the Library of Congress. As stated in Public Law 91-510 (2 USC 166 (d)(6)), one of the duties of CRS is \"to prepare summaries and digests of bills and resolutions of a public general nature introduced in the Senate or House of Representatives\". For more information, refer to the User Guide that accompanies this file."
        }
    }
}

"""

In [142]:
input_json = json.loads(input_json)

In [143]:
conformed_json = conform_data(input_json['billStatus'])

In [144]:
conformed_json['bill']['actions']

{'item': [{'actionDate': '2023-03-08',
   'text': 'Motion to reconsider laid on the table Agreed to without objection.',
   'type': 'Floor',
   'actionCode': 'H38310',
   'recordedVotes': []},
  {'actionDate': '2023-03-08',
   'text': 'On agreeing to the resolution Failed by the Yeas and Nays: 103 - 321 (Roll no. 136).',
   'type': 'Floor',
   'actionCode': 'H37100',
   'recordedVotes': [{'rollNumber': '136',
     'url': 'https://clerk.house.gov/evs/2023/roll136.xml',
     'chamber': 'House',
     'congress': '118',
     'date': '2023-03-08T23:00:41Z',
     'sessionNumber': '1'}]},
  {'actionDate': '2023-03-08',
   'text': 'Failed of passage/not agreed to in House On agreeing to the resolution Failed by the Yeas and Nays: 103 - 321 (Roll no. 136).',
   'type': 'Floor',
   'actionCode': '9000',
   'recordedVotes': [{'rollNumber': '136',
     'url': 'https://clerk.house.gov/evs/2023/roll136.xml',
     'chamber': 'House',
     'congress': '118',
     'date': '2023-03-08T23:00:41Z',
     '

In [145]:
def get_votes(bill_json, bill_type, save_folder):
    
    bill_actions = [action for action in bill_json['bill']['actions']['item']]
    recorded_votes = [vote for action in bill_actions for vote in action['recordedVotes']]
    return recorded_votes

In [146]:
get_votes(conformed_json, bill_type=None, folder=None)

TypeError: get_votes() got an unexpected keyword argument 'folder'

In [147]:
def conform_votes(vote_data, bill_type, bill_number, roll_call_number, chamber):
    conform_data = {
        'vote': get_vote_metadata(data=vote_data, chamber=chamber, bill_type=bill_type, bill_number=bill_number),
        'legislator': [legislator_dict(item, chamber) for item in get_raw_votes(vote_data, chamber=chamber)]
    }
    
    return conform_data

def get_vote_metadata(data, chamber, bill_type, bill_number):
    
    if chamber == 'House':
        vote_date = data['rollcall-vote']['vote-metadata']['action-date']
        date_obj = datetime.strptime(vote_date, '%d-%b-%Y')
        formatted_vote_date = date_obj.strftime('%Y/%m/%d')
        
        return {'date': formatted_vote_date,
                'bill_type': bill_type,
                'bill_number': bill_number,
                'roll_call_number': data['rollcall-vote']['vote-metadata']['rollcall-num'],
                'result': data['rollcall-vote']['vote-metadata']['vote-result'],
                'totals': {'yea': data['rollcall-vote']['vote-metadata']['vote-totals']['totals-by-vote']['yea-total'],
                           'nay': data['rollcall-vote']['vote-metadata']['vote-totals']['totals-by-vote']['nay-total'],
                           'present': data['rollcall-vote']['vote-metadata']['vote-totals']['totals-by-vote']['present-total'],
                           'not_voting': data['rollcall-vote']['vote-metadata']['vote-totals']['totals-by-vote']['not-voting-total']
                          }
                }
        
    elif chamber == 'Senate':
        vote_date = data['roll_call_vote']['vote_date']
        vote_date = vote_date.replace(': ', ':')
        date_obj = datetime.strptime(vote_date, '%B %d, %Y, %I:%M %p')
        formatted_vote_date = date_obj.strftime('%Y/%m/%d')
        
        return {'date': formatted_vote_date,
                'bill_type': bill_type,
                'bill_number': bill_number,
                'roll_call_number': data['roll_call_vote']['vote_number'],
                'result': data['roll_call_vote']['vote_result'],
                'totals': {'yea': data['roll_call_vote']['count']['yeas'],
                           'nay': data['roll_call_vote']['count']['nays'],
                           'present': data['roll_call_vote']['count']['present'],
                           'not_voting': data['roll_call_vote']['count']['absent']}
               }
                
   
    else:
        return {}
    
def get_raw_votes(data, chamber):
    if chamber == 'House':
        return data['rollcall-vote']['vote-data']['recorded-vote']
    if chamber == 'Senate':
        return data['roll_call_vote']['members']['member']
    return {}
        

def legislator_dict(item, chamber):
        if chamber == 'House':
            if isinstance(item, dict):
                return {
                    'id': item.get("legislator").get("@name-id"),
                    'first_name': None,
                    'last_name': item.get("legislator").get("@unaccented-name"),
                    'party': item.get("legislator").get("@party"),
                    'state': item.get("legislator").get("@state"),
                    'vote' : item.get("vote")
                    
                }
            return {}
        
        if chamber == 'Senate':
            if isinstance(item, dict):
                return {
                    'id': item.get("lis_member_id"),
                    'first_name': item.get("first_name"),
                    'last_name': item.get("last_name"),
                    'party': item.get("party"),
                    'state': item.get("state"),
                    'vote' : item.get("vote_cast")
                }
            return {}

In [132]:
conform_votes(vote_data=house_vote, bill_type='hr', bill_number=1, roll_call_number=1, chamber='House')

{'vote': {'date': '2023/03/30',
  'bill_type': 'hr',
  'bill_number': 1,
  'roll_call_number': '182',
  'result': 'Passed',
  'totals': {'yea': '225', 'nay': '204', 'present': '0', 'not_voting': '6'}},
 'legislator': [{'id': 'A000370',
   'first_name': None,
   'last_name': 'Adams',
   'party': 'D',
   'state': 'NC',
   'vote': 'Nay'},
  {'id': 'A000055',
   'first_name': None,
   'last_name': 'Aderholt',
   'party': 'R',
   'state': 'AL',
   'vote': 'Yea'},
  {'id': 'A000371',
   'first_name': None,
   'last_name': 'Aguilar',
   'party': 'D',
   'state': 'CA',
   'vote': 'Nay'},
  {'id': 'A000379',
   'first_name': None,
   'last_name': 'Alford',
   'party': 'R',
   'state': 'MO',
   'vote': 'Yea'},
  {'id': 'A000372',
   'first_name': None,
   'last_name': 'Allen',
   'party': 'R',
   'state': 'GA',
   'vote': 'Yea'},
  {'id': 'A000376',
   'first_name': None,
   'last_name': 'Allred',
   'party': 'D',
   'state': 'TX',
   'vote': 'Nay'},
  {'id': 'A000369',
   'first_name': None,
  

In [94]:
get_votes(conformed_json, save_folder='./')

In [96]:
! rm HCONRES_136.json 

In [122]:
senate_vote = {'roll_call_vote': {'congress': '118', 'session': '1', 'congress_year': '2023', 'vote_number': '2', 'vote_date': 'January 26, 2023, 01: 40 PM', 'modify_date': 'January 26, 2023,02: 47 PM', 'vote_question_text': 'On the Resolution S.Res. 13', 'vote_document_text': 'A resolution raising awareness and encouraging the prevention of stalking by designating January 2023 as "National Stalking Awareness Month".', 'vote_result_text': 'Resolution Agreed to (94-0)', 'question': 'On the Resolution', 'vote_title': 'S.Res. 13', 'majority_requirement': '1/2', 'vote_result': 'Resolution Agreed to', 'document': {'document_congress': '118', 'document_type': 'S.Res.', 'document_number': '13', 'document_name': 'S.Res. 13', 'document_title': 'A resolution raising awareness and encouraging the prevention of stalking by designating January 2023 as "National Stalking Awareness Month".', 'document_short_title': None
        }, 'amendment': {'amendment_number': None, 'amendment_to_amendment_number': None, 'amendment_to_amendment_to_amendment_number': None, 'amendment_to_document_number': None, 'amendment_to_document_short_title': None, 'amendment_purpose': 'No Statement of Purpose on File.'
        }, 'count': {'yeas': '94', 'nays': '0', 'present': None, 'absent': '6'
        }, 'tie_breaker': {'by_whom': None, 'tie_breaker_vote': None
        }, 'members': {'member': [
                {'member_full': 'Baldwin (D-WI)', 'last_name': 'Baldwin', 'first_name': 'Tammy', 'party': 'D', 'state': 'WI', 'vote_cast': 'Yea', 'lis_member_id': 'S354'
                },
                {'member_full': 'Barrasso (R-WY)', 'last_name': 'Barrasso', 'first_name': 'John', 'party': 'R', 'state': 'WY', 'vote_cast': 'Yea', 'lis_member_id': 'S317'
                },
                {'member_full': 'Bennet (D-CO)', 'last_name': 'Bennet', 'first_name': 'Michael', 'party': 'D', 'state': 'CO', 'vote_cast': 'Yea', 'lis_member_id': 'S330'
                },
                {'member_full': 'Blackburn (R-TN)', 'last_name': 'Blackburn', 'first_name': 'Marsha', 'party': 'R', 'state': 'TN', 'vote_cast': 'Yea', 'lis_member_id': 'S396'
                },
                {'member_full': 'Blumenthal (D-CT)', 'last_name': 'Blumenthal', 'first_name': 'Richard', 'party': 'D', 'state': 'CT', 'vote_cast': 'Yea', 'lis_member_id': 'S341'
                },
                {'member_full': 'Booker (D-NJ)', 'last_name': 'Booker', 'first_name': 'Cory', 'party': 'D', 'state': 'NJ', 'vote_cast': 'Yea', 'lis_member_id': 'S370'
                },
                {'member_full': 'Boozman (R-AR)', 'last_name': 'Boozman', 'first_name': 'John', 'party': 'R', 'state': 'AR', 'vote_cast': 'Yea', 'lis_member_id': 'S343'
                },
                {'member_full': 'Braun (R-IN)', 'last_name': 'Braun', 'first_name': 'Mike', 'party': 'R', 'state': 'IN', 'vote_cast': 'Yea', 'lis_member_id': 'S397'
                },
                {'member_full': 'Britt (R-AL)', 'last_name': 'Britt', 'first_name': 'Katie', 'party': 'R', 'state': 'AL', 'vote_cast': 'Yea', 'lis_member_id': 'S416'
                },
                {'member_full': 'Brown (D-OH)', 'last_name': 'Brown', 'first_name': 'Sherrod', 'party': 'D', 'state': 'OH', 'vote_cast': 'Yea', 'lis_member_id': 'S307'
                },
                {'member_full': 'Budd (R-NC)', 'last_name': 'Budd', 'first_name': 'Ted', 'party': 'R', 'state': 'NC', 'vote_cast': 'Yea', 'lis_member_id': 'S417'
                },
                {'member_full': 'Cantwell (D-WA)', 'last_name': 'Cantwell', 'first_name': 'Maria', 'party': 'D', 'state': 'WA', 'vote_cast': 'Yea', 'lis_member_id': 'S275'
                },
                {'member_full': 'Capito (R-WV)', 'last_name': 'Capito', 'first_name': 'Shelley', 'party': 'R', 'state': 'WV', 'vote_cast': 'Yea', 'lis_member_id': 'S372'
                },
                {'member_full': 'Cardin (D-MD)', 'last_name': 'Cardin', 'first_name': 'Ben', 'party': 'D', 'state': 'MD', 'vote_cast': 'Yea', 'lis_member_id': 'S308'
                },
                {'member_full': 'Carper (D-DE)', 'last_name': 'Carper', 'first_name': 'Thomas', 'party': 'D', 'state': 'DE', 'vote_cast': 'Yea', 'lis_member_id': 'S277'
                },
                {'member_full': 'Casey (D-PA)', 'last_name': 'Casey', 'first_name': 'Bob', 'party': 'D', 'state': 'PA', 'vote_cast': 'Yea', 'lis_member_id': 'S309'
                },
                {'member_full': 'Cassidy (R-LA)', 'last_name': 'Cassidy', 'first_name': 'Bill', 'party': 'R', 'state': 'LA', 'vote_cast': 'Yea', 'lis_member_id': 'S373'
                },
                {'member_full': 'Collins (R-ME)', 'last_name': 'Collins', 'first_name': 'Susan', 'party': 'R', 'state': 'ME', 'vote_cast': 'Yea', 'lis_member_id': 'S252'
                },
                {'member_full': 'Coons (D-DE)', 'last_name': 'Coons', 'first_name': 'Christopher', 'party': 'D', 'state': 'DE', 'vote_cast': 'Yea', 'lis_member_id': 'S337'
                },
                {'member_full': 'Cornyn (R-TX)', 'last_name': 'Cornyn', 'first_name': 'John', 'party': 'R', 'state': 'TX', 'vote_cast': 'Yea', 'lis_member_id': 'S287'
                },
                {'member_full': 'Cortez Masto (D-NV)', 'last_name': 'Cortez Masto', 'first_name': 'Catherine', 'party': 'D', 'state': 'NV', 'vote_cast': 'Yea', 'lis_member_id': 'S385'
                },
                {'member_full': 'Cotton (R-AR)', 'last_name': 'Cotton', 'first_name': 'Tom', 'party': 'R', 'state': 'AR', 'vote_cast': 'Yea', 'lis_member_id': 'S374'
                },
                {'member_full': 'Cramer (R-ND)', 'last_name': 'Cramer', 'first_name': 'Kevin', 'party': 'R', 'state': 'ND', 'vote_cast': 'Yea', 'lis_member_id': 'S398'
                },
                {'member_full': 'Crapo (R-ID)', 'last_name': 'Crapo', 'first_name': 'Mike', 'party': 'R', 'state': 'ID', 'vote_cast': 'Yea', 'lis_member_id': 'S266'
                },
                {'member_full': 'Cruz (R-TX)', 'last_name': 'Cruz', 'first_name': 'Ted', 'party': 'R', 'state': 'TX', 'vote_cast': 'Yea', 'lis_member_id': 'S355'
                },
                {'member_full': 'Daines (R-MT)', 'last_name': 'Daines', 'first_name': 'Steve', 'party': 'R', 'state': 'MT', 'vote_cast': 'Yea', 'lis_member_id': 'S375'
                },
                {'member_full': 'Duckworth (D-IL)', 'last_name': 'Duckworth', 'first_name': 'Tammy', 'party': 'D', 'state': 'IL', 'vote_cast': 'Yea', 'lis_member_id': 'S386'
                },
                {'member_full': 'Durbin (D-IL)', 'last_name': 'Durbin', 'first_name': 'Richard', 'party': 'D', 'state': 'IL', 'vote_cast': 'Yea', 'lis_member_id': 'S253'
                },
                {'member_full': 'Ernst (R-IA)', 'last_name': 'Ernst', 'first_name': 'Joni', 'party': 'R', 'state': 'IA', 'vote_cast': 'Yea', 'lis_member_id': 'S376'
                },
                {'member_full': 'Feinstein (D-CA)', 'last_name': 'Feinstein', 'first_name': 'Dianne', 'party': 'D', 'state': 'CA', 'vote_cast': 'Not Voting', 'lis_member_id': 'S221'
                },
                {'member_full': 'Fetterman (D-PA)', 'last_name': 'Fetterman', 'first_name': 'John', 'party': 'D', 'state': 'PA', 'vote_cast': 'Yea', 'lis_member_id': 'S418'
                },
                {'member_full': 'Fischer (R-NE)', 'last_name': 'Fischer', 'first_name': 'Deb', 'party': 'R', 'state': 'NE', 'vote_cast': 'Yea', 'lis_member_id': 'S357'
                },
                {'member_full': 'Gillibrand (D-NY)', 'last_name': 'Gillibrand', 'first_name': 'Kirsten', 'party': 'D', 'state': 'NY', 'vote_cast': 'Yea', 'lis_member_id': 'S331'
                },
                {'member_full': 'Graham (R-SC)', 'last_name': 'Graham', 'first_name': 'Lindsey', 'party': 'R', 'state': 'SC', 'vote_cast': 'Not Voting', 'lis_member_id': 'S293'
                },
                {'member_full': 'Grassley (R-IA)', 'last_name': 'Grassley', 'first_name': 'Chuck', 'party': 'R', 'state': 'IA', 'vote_cast': 'Yea', 'lis_member_id': 'S153'
                },
                {'member_full': 'Hagerty (R-TN)', 'last_name': 'Hagerty', 'first_name': 'Bill', 'party': 'R', 'state': 'TN', 'vote_cast': 'Yea', 'lis_member_id': 'S407'
                },
                {'member_full': 'Hassan (D-NH)', 'last_name': 'Hassan', 'first_name': 'Maggie', 'party': 'D', 'state': 'NH', 'vote_cast': 'Yea', 'lis_member_id': 'S388'
                },
                {'member_full': 'Hawley (R-MO)', 'last_name': 'Hawley', 'first_name': 'Josh', 'party': 'R', 'state': 'MO', 'vote_cast': 'Yea', 'lis_member_id': 'S399'
                },
                {'member_full': 'Heinrich (D-NM)', 'last_name': 'Heinrich', 'first_name': 'Martin', 'party': 'D', 'state': 'NM', 'vote_cast': 'Yea', 'lis_member_id': 'S359'
                },
                {'member_full': 'Hickenlooper (D-CO)', 'last_name': 'Hickenlooper', 'first_name': 'John', 'party': 'D', 'state': 'CO', 'vote_cast': 'Yea', 'lis_member_id': 'S408'
                },
                {'member_full': 'Hirono (D-HI)', 'last_name': 'Hirono', 'first_name': 'Mazie', 'party': 'D', 'state': 'HI', 'vote_cast': 'Yea', 'lis_member_id': 'S361'
                },
                {'member_full': 'Hoeven (R-ND)', 'last_name': 'Hoeven', 'first_name': 'John', 'party': 'R', 'state': 'ND', 'vote_cast': 'Yea', 'lis_member_id': 'S344'
                },
                {'member_full': 'Hyde-Smith (R-MS)', 'last_name': 'Hyde-Smith', 'first_name': 'Cindy', 'party': 'R', 'state': 'MS', 'vote_cast': 'Yea', 'lis_member_id': 'S395'
                },
                {'member_full': 'Johnson (R-WI)', 'last_name': 'Johnson', 'first_name': 'Ron', 'party': 'R', 'state': 'WI', 'vote_cast': 'Not Voting', 'lis_member_id': 'S345'
                },
                {'member_full': 'Kaine (D-VA)', 'last_name': 'Kaine', 'first_name': 'Timothy', 'party': 'D', 'state': 'VA', 'vote_cast': 'Yea', 'lis_member_id': 'S362'
                },
                {'member_full': 'Kelly (D-AZ)', 'last_name': 'Kelly', 'first_name': 'Mark', 'party': 'D', 'state': 'AZ', 'vote_cast': 'Yea', 'lis_member_id': 'S406'
                },
                {'member_full': 'Kennedy (R-LA)', 'last_name': 'Kennedy', 'first_name': 'John', 'party': 'R', 'state': 'LA', 'vote_cast': 'Yea', 'lis_member_id': 'S389'
                },
                {'member_full': 'King (I-ME)', 'last_name': 'King', 'first_name': 'Angus', 'party': 'I', 'state': 'ME', 'vote_cast': 'Yea', 'lis_member_id': 'S363'
                },
                {'member_full': 'Klobuchar (D-MN)', 'last_name': 'Klobuchar', 'first_name': 'Amy', 'party': 'D', 'state': 'MN', 'vote_cast': 'Yea', 'lis_member_id': 'S311'
                },
                {'member_full': 'Lankford (R-OK)', 'last_name': 'Lankford', 'first_name': 'James', 'party': 'R', 'state': 'OK', 'vote_cast': 'Yea', 'lis_member_id': 'S378'
                },
                {'member_full': 'Lee (R-UT)', 'last_name': 'Lee', 'first_name': 'Mike', 'party': 'R', 'state': 'UT', 'vote_cast': 'Yea', 'lis_member_id': 'S346'
                },
                {'member_full': 'Lujan (D-NM)', 'last_name': 'Lujan', 'first_name': 'Ben', 'party': 'D', 'state': 'NM', 'vote_cast': 'Yea', 'lis_member_id': 'S409'
                },
                {'member_full': 'Lummis (R-WY)', 'last_name': 'Lummis', 'first_name': 'Cynthia', 'party': 'R', 'state': 'WY', 'vote_cast': 'Yea', 'lis_member_id': 'S410'
                },
                {'member_full': 'Manchin (D-WV)', 'last_name': 'Manchin', 'first_name': 'Joseph', 'party': 'D', 'state': 'WV', 'vote_cast': 'Yea', 'lis_member_id': 'S338'
                },
                {'member_full': 'Markey (D-MA)', 'last_name': 'Markey', 'first_name': 'Edward', 'party': 'D', 'state': 'MA', 'vote_cast': 'Yea', 'lis_member_id': 'S369'
                },
                {'member_full': 'Marshall (R-KS)', 'last_name': 'Marshall', 'first_name': 'Roger', 'party': 'R', 'state': 'KS', 'vote_cast': 'Yea', 'lis_member_id': 'S411'
                },
                {'member_full': 'McConnell (R-KY)', 'last_name': 'McConnell', 'first_name': 'Mitch', 'party': 'R', 'state': 'KY', 'vote_cast': 'Yea', 'lis_member_id': 'S174'
                },
                {'member_full': 'Menendez (D-NJ)', 'last_name': 'Menendez', 'first_name': 'Robert', 'party': 'D', 'state': 'NJ', 'vote_cast': 'Yea', 'lis_member_id': 'S306'
                },
                {'member_full': 'Merkley (D-OR)', 'last_name': 'Merkley', 'first_name': 'Jeff', 'party': 'D', 'state': 'OR', 'vote_cast': 'Yea', 'lis_member_id': 'S322'
                },
                {'member_full': 'Moran (R-KS)', 'last_name': 'Moran', 'first_name': 'Jerry', 'party': 'R', 'state': 'KS', 'vote_cast': 'Not Voting', 'lis_member_id': 'S347'
                },
                {'member_full': 'Mullin (R-OK)', 'last_name': 'Mullin', 'first_name': 'Markwayne', 'party': 'R', 'state': 'OK', 'vote_cast': 'Yea', 'lis_member_id': 'S419'
                },
                {'member_full': 'Murkowski (R-AK)', 'last_name': 'Murkowski', 'first_name': 'Lisa', 'party': 'R', 'state': 'AK', 'vote_cast': 'Yea', 'lis_member_id': 'S288'
                },
                {'member_full': 'Murphy (D-CT)', 'last_name': 'Murphy', 'first_name': 'Christopher', 'party': 'D', 'state': 'CT', 'vote_cast': 'Yea', 'lis_member_id': 'S364'
                },
                {'member_full': 'Murray (D-WA)', 'last_name': 'Murray', 'first_name': 'Patty', 'party': 'D', 'state': 'WA', 'vote_cast': 'Yea', 'lis_member_id': 'S229'
                },
                {'member_full': 'Ossoff (D-GA)', 'last_name': 'Ossoff', 'first_name': 'Jon', 'party': 'D', 'state': 'GA', 'vote_cast': 'Yea', 'lis_member_id': 'S414'
                },
                {'member_full': 'Padilla (D-CA)', 'last_name': 'Padilla', 'first_name': 'Alex', 'party': 'D', 'state': 'CA', 'vote_cast': 'Yea', 'lis_member_id': 'S413'
                },
                {'member_full': 'Paul (R-KY)', 'last_name': 'Paul', 'first_name': 'Rand', 'party': 'R', 'state': 'KY', 'vote_cast': 'Not Voting', 'lis_member_id': 'S348'
                },
                {'member_full': 'Peters (D-MI)', 'last_name': 'Peters', 'first_name': 'Gary', 'party': 'D', 'state': 'MI', 'vote_cast': 'Yea', 'lis_member_id': 'S380'
                },
                {'member_full': 'Reed (D-RI)', 'last_name': 'Reed', 'first_name': 'John', 'party': 'D', 'state': 'RI', 'vote_cast': 'Yea', 'lis_member_id': 'S259'
                },
                {'member_full': 'Ricketts (R-NE)', 'last_name': 'Ricketts', 'first_name': 'Pete', 'party': 'R', 'state': 'NE', 'vote_cast': 'Yea', 'lis_member_id': 'S423'
                },
                {'member_full': 'Risch (R-ID)', 'last_name': 'Risch', 'first_name': 'James', 'party': 'R', 'state': 'ID', 'vote_cast': 'Yea', 'lis_member_id': 'S323'
                },
                {'member_full': 'Romney (R-UT)', 'last_name': 'Romney', 'first_name': 'Mitt', 'party': 'R', 'state': 'UT', 'vote_cast': 'Yea', 'lis_member_id': 'S401'
                },
                {'member_full': 'Rosen (D-NV)', 'last_name': 'Rosen', 'first_name': 'Jacklyn', 'party': 'D', 'state': 'NV', 'vote_cast': 'Yea', 'lis_member_id': 'S402'
                },
                {'member_full': 'Rounds (R-SD)', 'last_name': 'Rounds', 'first_name': 'Mike', 'party': 'R', 'state': 'SD', 'vote_cast': 'Yea', 'lis_member_id': 'S381'
                },
                {'member_full': 'Rubio (R-FL)', 'last_name': 'Rubio', 'first_name': 'Marco', 'party': 'R', 'state': 'FL', 'vote_cast': 'Yea', 'lis_member_id': 'S350'
                },
                {'member_full': 'Sanders (I-VT)', 'last_name': 'Sanders', 'first_name': 'Bernard', 'party': 'I', 'state': 'VT', 'vote_cast': 'Yea', 'lis_member_id': 'S313'
                },
                {'member_full': 'Schatz (D-HI)', 'last_name': 'Schatz', 'first_name': 'Brian', 'party': 'D', 'state': 'HI', 'vote_cast': 'Yea', 'lis_member_id': 'S353'
                },
                {'member_full': 'Schmitt (R-MO)', 'last_name': 'Schmitt', 'first_name': 'Eric', 'party': 'R', 'state': 'MO', 'vote_cast': 'Yea', 'lis_member_id': 'S420'
                },
                {'member_full': 'Schumer (D-NY)', 'last_name': 'Schumer', 'first_name': 'Charles', 'party': 'D', 'state': 'NY', 'vote_cast': 'Yea', 'lis_member_id': 'S270'
                },
                {'member_full': 'Scott (R-FL)', 'last_name': 'Scott', 'first_name': 'Rick', 'party': 'R', 'state': 'FL', 'vote_cast': 'Yea', 'lis_member_id': 'S404'
                },
                {'member_full': 'Scott (R-SC)', 'last_name': 'Scott', 'first_name': 'Tim', 'party': 'R', 'state': 'SC', 'vote_cast': 'Yea', 'lis_member_id': 'S365'
                },
                {'member_full': 'Shaheen (D-NH)', 'last_name': 'Shaheen', 'first_name': 'Jeanne', 'party': 'D', 'state': 'NH', 'vote_cast': 'Yea', 'lis_member_id': 'S324'
                },
                {'member_full': 'Sinema (I-AZ)', 'last_name': 'Sinema', 'first_name': 'Kyrsten', 'party': 'I', 'state': 'AZ', 'vote_cast': 'Yea', 'lis_member_id': 'S403'
                },
                {'member_full': 'Smith (D-MN)', 'last_name': 'Smith', 'first_name': 'Tina', 'party': 'D', 'state': 'MN', 'vote_cast': 'Yea', 'lis_member_id': 'S394'
                },
                {'member_full': 'Stabenow (D-MI)', 'last_name': 'Stabenow', 'first_name': 'Debbie', 'party': 'D', 'state': 'MI', 'vote_cast': 'Yea', 'lis_member_id': 'S284'
                },
                {'member_full': 'Sullivan (R-AK)', 'last_name': 'Sullivan', 'first_name': 'Dan', 'party': 'R', 'state': 'AK', 'vote_cast': 'Yea', 'lis_member_id': 'S383'
                },
                {'member_full': 'Tester (D-MT)', 'last_name': 'Tester', 'first_name': 'Jon', 'party': 'D', 'state': 'MT', 'vote_cast': 'Yea', 'lis_member_id': 'S314'
                },
                {'member_full': 'Thune (R-SD)', 'last_name': 'Thune', 'first_name': 'John', 'party': 'R', 'state': 'SD', 'vote_cast': 'Yea', 'lis_member_id': 'S303'
                },
                {'member_full': 'Tillis (R-NC)', 'last_name': 'Tillis', 'first_name': 'Thomas', 'party': 'R', 'state': 'NC', 'vote_cast': 'Yea', 'lis_member_id': 'S384'
                },
                {'member_full': 'Tuberville (R-AL)', 'last_name': 'Tuberville', 'first_name': 'Tommy', 'party': 'R', 'state': 'AL', 'vote_cast': 'Yea', 'lis_member_id': 'S412'
                },
                {'member_full': 'Van Hollen (D-MD)', 'last_name': 'Van Hollen', 'first_name': 'Chris', 'party': 'D', 'state': 'MD', 'vote_cast': 'Yea', 'lis_member_id': 'S390'
                },
                {'member_full': 'Vance (R-OH)', 'last_name': 'Vance', 'first_name': 'J.', 'party': 'R', 'state': 'OH', 'vote_cast': 'Not Voting', 'lis_member_id': 'S421'
                },
                {'member_full': 'Warner (D-VA)', 'last_name': 'Warner', 'first_name': 'Mark', 'party': 'D', 'state': 'VA', 'vote_cast': 'Yea', 'lis_member_id': 'S327'
                },
                {'member_full': 'Warnock (D-GA)', 'last_name': 'Warnock', 'first_name': 'Raphael', 'party': 'D', 'state': 'GA', 'vote_cast': 'Yea', 'lis_member_id': 'S415'
                },
                {'member_full': 'Warren (D-MA)', 'last_name': 'Warren', 'first_name': 'Elizabeth', 'party': 'D', 'state': 'MA', 'vote_cast': 'Yea', 'lis_member_id': 'S366'
                },
                {'member_full': 'Welch (D-VT)', 'last_name': 'Welch', 'first_name': 'Peter', 'party': 'D', 'state': 'VT', 'vote_cast': 'Yea', 'lis_member_id': 'S422'
                },
                {'member_full': 'Whitehouse (D-RI)', 'last_name': 'Whitehouse', 'first_name': 'Sheldon', 'party': 'D', 'state': 'RI', 'vote_cast': 'Yea', 'lis_member_id': 'S316'
                },
                {'member_full': 'Wicker (R-MS)', 'last_name': 'Wicker', 'first_name': 'Roger', 'party': 'R', 'state': 'MS', 'vote_cast': 'Yea', 'lis_member_id': 'S318'
                },
                {'member_full': 'Wyden (D-OR)', 'last_name': 'Wyden', 'first_name': 'Ron', 'party': 'D', 'state': 'OR', 'vote_cast': 'Yea', 'lis_member_id': 'S247'
                },
                {'member_full': 'Young (R-IN)', 'last_name': 'Young', 'first_name': 'Todd', 'party': 'R', 'state': 'IN', 'vote_cast': 'Yea', 'lis_member_id': 'S391'
                }
            ]
        }
    }
}

In [130]:
house_vote = {'rollcall-vote': {'vote-metadata': {'majority': 'R', 'congress': '118', 'session': '1st', 'chamber': 'U.S. House of Representatives', 'rollcall-num': '182', 'legis-num': 'H R 1', 'vote-question': 'On Passage', 'vote-type': 'YEA-AND-NAY', 'vote-result': 'Passed', 'action-date': '30-Mar-2023', 'action-time': {'@time-etz': '11: 43', '#text': '11: 43 AM'
            }, 'vote-desc': 'Lower Energy Costs Act', 'vote-totals': {'totals-by-party-header': {'party-header': 'Party', 'yea-header': 'Yeas', 'nay-header': 'Nays', 'present-header': 'Answered “Present”', 'not-voting-header': 'Not Voting'
                }, 'totals-by-party': [
                    {'party': 'Republican', 'yea-total': '221', 'nay-total': '1', 'present-total': '0', 'not-voting-total': '0'
                    },
                    {'party': 'Democratic', 'yea-total': '4', 'nay-total': '203', 'present-total': '0', 'not-voting-total': '6'
                    },
                    {'party': 'Independent', 'yea-total': '0', 'nay-total': '0', 'present-total': '0', 'not-voting-total': '0'
                    }
                ], 'totals-by-vote': {'total-stub': 'Totals', 'yea-total': '225', 'nay-total': '204', 'present-total': '0', 'not-voting-total': '6'
                }
            }
        }, 'vote-data': {'recorded-vote': [
                {'legislator': {'@name-id': 'A000370', '@sort-field': 'Adams', '@unaccented-name': 'Adams', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Adams'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'A000055', '@sort-field': 'Aderholt', '@unaccented-name': 'Aderholt', '@party': 'R', '@state': 'AL', '@role': 'legislator', '#text': 'Aderholt'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'A000371', '@sort-field': 'Aguilar', '@unaccented-name': 'Aguilar', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Aguilar'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'A000379', '@sort-field': 'Alford', '@unaccented-name': 'Alford', '@party': 'R', '@state': 'MO', '@role': 'legislator', '#text': 'Alford'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'A000372', '@sort-field': 'Allen', '@unaccented-name': 'Allen', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Allen'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'A000376', '@sort-field': 'Allred', '@unaccented-name': 'Allred', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Allred'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'A000369', '@sort-field': 'Amodei', '@unaccented-name': 'Amodei', '@party': 'R', '@state': 'NV', '@role': 'legislator', '#text': 'Amodei'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'A000377', '@sort-field': 'Armstrong', '@unaccented-name': 'Armstrong', '@party': 'R', '@state': 'ND', '@role': 'legislator', '#text': 'Armstrong'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'A000375', '@sort-field': 'Arrington', '@unaccented-name': 'Arrington', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Arrington'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'A000148', '@sort-field': 'Auchincloss', '@unaccented-name': 'Auchincloss', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Auchincloss'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001291', '@sort-field': 'Babin', '@unaccented-name': 'Babin', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Babin'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001298', '@sort-field': 'Bacon', '@unaccented-name': 'Bacon', '@party': 'R', '@state': 'NE', '@role': 'legislator', '#text': 'Bacon'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001307', '@sort-field': 'Baird', '@unaccented-name': 'Baird', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Baird'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001306', '@sort-field': 'Balderson', '@unaccented-name': 'Balderson', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Balderson'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001318', '@sort-field': 'Balint', '@unaccented-name': 'Balint', '@party': 'D', '@state': 'VT', '@role': 'legislator', '#text': 'Balint'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001299', '@sort-field': 'Banks', '@unaccented-name': 'Banks', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Banks'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001282', '@sort-field': 'Barr', '@unaccented-name': 'Barr', '@party': 'R', '@state': 'KY', '@role': 'legislator', '#text': 'Barr'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001300', '@sort-field': 'Barragan', '@unaccented-name': 'Barragan', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Barragán'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001314', '@sort-field': 'Bean (FL)', '@unaccented-name': 'Bean (FL)', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Bean (FL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001281', '@sort-field': 'Beatty', '@unaccented-name': 'Beatty', '@party': 'D', '@state': 'OH', '@role': 'legislator', '#text': 'Beatty'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B000668', '@sort-field': 'Bentz', '@unaccented-name': 'Bentz', '@party': 'R', '@state': 'OR', '@role': 'legislator', '#text': 'Bentz'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001287', '@sort-field': 'Bera', '@unaccented-name': 'Bera', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Bera'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001301', '@sort-field': 'Bergman', '@unaccented-name': 'Bergman', '@party': 'R', '@state': 'MI', '@role': 'legislator', '#text': 'Bergman'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001292', '@sort-field': 'Beyer', '@unaccented-name': 'Beyer', '@party': 'D', '@state': 'VA', '@role': 'legislator', '#text': 'Beyer'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B000740', '@sort-field': 'Bice', '@unaccented-name': 'Bice', '@party': 'R', '@state': 'OK', '@role': 'legislator', '#text': 'Bice'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001302', '@sort-field': 'Biggs', '@unaccented-name': 'Biggs', '@party': 'R', '@state': 'AZ', '@role': 'legislator', '#text': 'Biggs'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001257', '@sort-field': 'Bilirakis', '@unaccented-name': 'Bilirakis', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Bilirakis'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B000490', '@sort-field': 'Bishop (GA)', '@unaccented-name': 'Bishop (GA)', '@party': 'D', '@state': 'GA', '@role': 'legislator', '#text': 'Bishop (GA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001311', '@sort-field': 'Bishop (NC)', '@unaccented-name': 'Bishop (NC)', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'Bishop (NC)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B000574', '@sort-field': 'Blumenauer', '@unaccented-name': 'Blumenauer', '@party': 'D', '@state': 'OR', '@role': 'legislator', '#text': 'Blumenauer'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001303', '@sort-field': 'Blunt Rochester', '@unaccented-name': 'Blunt Rochester', '@party': 'D', '@state': 'DE', '@role': 'legislator', '#text': 'Blunt Rochester'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B000825', '@sort-field': 'Boebert', '@unaccented-name': 'Boebert', '@party': 'R', '@state': 'CO', '@role': 'legislator', '#text': 'Boebert'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001278', '@sort-field': 'Bonamici', '@unaccented-name': 'Bonamici', '@party': 'D', '@state': 'OR', '@role': 'legislator', '#text': 'Bonamici'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001295', '@sort-field': 'Bost', '@unaccented-name': 'Bost', '@party': 'R', '@state': 'IL', '@role': 'legislator', '#text': 'Bost'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001223', '@sort-field': 'Bowman', '@unaccented-name': 'Bowman', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Bowman'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001296', '@sort-field': 'Boyle (PA)', '@unaccented-name': 'Boyle (PA)', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Boyle (PA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001317', '@sort-field': 'Brecheen', '@unaccented-name': 'Brecheen', '@party': 'R', '@state': 'OK', '@role': 'legislator', '#text': 'Brecheen'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001313', '@sort-field': 'Brown', '@unaccented-name': 'Brown', '@party': 'D', '@state': 'OH', '@role': 'legislator', '#text': 'Brown'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001285', '@sort-field': 'Brownley', '@unaccented-name': 'Brownley', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Brownley'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001260', '@sort-field': 'Buchanan', '@unaccented-name': 'Buchanan', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Buchanan'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001297', '@sort-field': 'Buck', '@unaccented-name': 'Buck', '@party': 'R', '@state': 'CO', '@role': 'legislator', '#text': 'Buck'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001275', '@sort-field': 'Bucshon', '@unaccented-name': 'Bucshon', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Bucshon'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001315', '@sort-field': 'Budzinski', '@unaccented-name': 'Budzinski', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Budzinski'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'B001309', '@sort-field': 'Burchett', '@unaccented-name': 'Burchett', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Burchett'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001248', '@sort-field': 'Burgess', '@unaccented-name': 'Burgess', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Burgess'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001316', '@sort-field': 'Burlison', '@unaccented-name': 'Burlison', '@party': 'R', '@state': 'MO', '@role': 'legislator', '#text': 'Burlison'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'B001224', '@sort-field': 'Bush', '@unaccented-name': 'Bush', '@party': 'D', '@state': 'MO', '@role': 'legislator', '#text': 'Bush'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C000059', '@sort-field': 'Calvert', '@unaccented-name': 'Calvert', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Calvert'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001039', '@sort-field': 'Cammack', '@unaccented-name': 'Cammack', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Cammack'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001134', '@sort-field': 'Caraveo', '@unaccented-name': 'Caraveo', '@party': 'D', '@state': 'CO', '@role': 'legislator', '#text': 'Caraveo'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001112', '@sort-field': 'Carbajal', '@unaccented-name': 'Carbajal', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Carbajal'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001097', '@sort-field': 'Cardenas', '@unaccented-name': 'Cardenas', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Cárdenas'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001126', '@sort-field': 'Carey', '@unaccented-name': 'Carey', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Carey'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001054', '@sort-field': 'Carl', '@unaccented-name': 'Carl', '@party': 'R', '@state': 'AL', '@role': 'legislator', '#text': 'Carl'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001072', '@sort-field': 'Carson', '@unaccented-name': 'Carson', '@party': 'D', '@state': 'IN', '@role': 'legislator', '#text': 'Carson'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001103', '@sort-field': 'Carter (GA)', '@unaccented-name': 'Carter (GA)', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Carter (GA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001125', '@sort-field': 'Carter (LA)', '@unaccented-name': 'Carter (LA)', '@party': 'D', '@state': 'LA', '@role': 'legislator', '#text': 'Carter (LA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001051', '@sort-field': 'Carter (TX)', '@unaccented-name': 'Carter (TX)', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Carter (TX)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001090', '@sort-field': 'Cartwright', '@unaccented-name': 'Cartwright', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Cartwright'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001131', '@sort-field': 'Casar', '@unaccented-name': 'Casar', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Casar'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001055', '@sort-field': 'Case', '@unaccented-name': 'Case', '@party': 'D', '@state': 'HI', '@role': 'legislator', '#text': 'Case'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001117', '@sort-field': 'Casten', '@unaccented-name': 'Casten', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Casten'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001066', '@sort-field': 'Castor (FL)', '@unaccented-name': 'Castor (FL)', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Castor (FL)'
                    }, 'vote': 'Not Voting'
                },
                {'legislator': {'@name-id': 'C001091', '@sort-field': 'Castro (TX)', '@unaccented-name': 'Castro (TX)', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Castro (TX)'
                    }, 'vote': 'Not Voting'
                },
                {'legislator': {'@name-id': 'C001135', '@sort-field': 'Chavez-DeRemer', '@unaccented-name': 'Chavez-DeRemer', '@party': 'R', '@state': 'OR', '@role': 'legislator', '#text': 'Chavez-DeRemer'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001127', '@sort-field': 'Cherfilus-McCormick', '@unaccented-name': 'Cherfilus-McCormick', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Cherfilus-McCormick'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001080', '@sort-field': 'Chu', '@unaccented-name': 'Chu', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Chu'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001084', '@sort-field': 'Cicilline', '@unaccented-name': 'Cicilline', '@party': 'D', '@state': 'RI', '@role': 'legislator', '#text': 'Cicilline'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001133', '@sort-field': 'Ciscomani', '@unaccented-name': 'Ciscomani', '@party': 'R', '@state': 'AZ', '@role': 'legislator', '#text': 'Ciscomani'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001101', '@sort-field': 'Clark (MA)', '@unaccented-name': 'Clark (MA)', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Clark (MA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001067', '@sort-field': 'Clarke (NY)', '@unaccented-name': 'Clarke (NY)', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Clarke (NY)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001061', '@sort-field': 'Cleaver', '@unaccented-name': 'Cleaver', '@party': 'D', '@state': 'MO', '@role': 'legislator', '#text': 'Cleaver'
                    }, 'vote': 'Not Voting'
                },
                {'legislator': {'@name-id': 'C001118', '@sort-field': 'Cline', '@unaccented-name': 'Cline', '@party': 'R', '@state': 'VA', '@role': 'legislator', '#text': 'Cline'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001115', '@sort-field': 'Cloud', '@unaccented-name': 'Cloud', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Cloud'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C000537', '@sort-field': 'Clyburn', '@unaccented-name': 'Clyburn', '@party': 'D', '@state': 'SC', '@role': 'legislator', '#text': 'Clyburn'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001116', '@sort-field': 'Clyde', '@unaccented-name': 'Clyde', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Clyde'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001068', '@sort-field': 'Cohen', '@unaccented-name': 'Cohen', '@party': 'D', '@state': 'TN', '@role': 'legislator', '#text': 'Cohen'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001053', '@sort-field': 'Cole', '@unaccented-name': 'Cole', '@party': 'R', '@state': 'OK', '@role': 'legislator', '#text': 'Cole'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001129', '@sort-field': 'Collins', '@unaccented-name': 'Collins', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Collins'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001108', '@sort-field': 'Comer', '@unaccented-name': 'Comer', '@party': 'R', '@state': 'KY', '@role': 'legislator', '#text': 'Comer'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001078', '@sort-field': 'Connolly', '@unaccented-name': 'Connolly', '@party': 'D', '@state': 'VA', '@role': 'legislator', '#text': 'Connolly'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001110', '@sort-field': 'Correa', '@unaccented-name': 'Correa', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Correa'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001059', '@sort-field': 'Costa', '@unaccented-name': 'Costa', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Costa'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001069', '@sort-field': 'Courtney', '@unaccented-name': 'Courtney', '@party': 'D', '@state': 'CT', '@role': 'legislator', '#text': 'Courtney'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001119', '@sort-field': 'Craig', '@unaccented-name': 'Craig', '@party': 'D', '@state': 'MN', '@role': 'legislator', '#text': 'Craig'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001132', '@sort-field': 'Crane', '@unaccented-name': 'Crane', '@party': 'R', '@state': 'AZ', '@role': 'legislator', '#text': 'Crane'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001087', '@sort-field': 'Crawford', '@unaccented-name': 'Crawford', '@party': 'R', '@state': 'AR', '@role': 'legislator', '#text': 'Crawford'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001120', '@sort-field': 'Crenshaw', '@unaccented-name': 'Crenshaw', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Crenshaw'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001130', '@sort-field': 'Crockett', '@unaccented-name': 'Crockett', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Crockett'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001121', '@sort-field': 'Crow', '@unaccented-name': 'Crow', '@party': 'D', '@state': 'CO', '@role': 'legislator', '#text': 'Crow'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'C001063', '@sort-field': 'Cuellar', '@unaccented-name': 'Cuellar', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Cuellar'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'C001114', '@sort-field': 'Curtis', '@unaccented-name': 'Curtis', '@party': 'R', '@state': 'UT', '@role': 'legislator', '#text': 'Curtis'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000632', '@sort-field': "D'Esposito", '@unaccented-name': "D'Esposito", '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': "D'Esposito"
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000629', '@sort-field': 'Davids (KS)', '@unaccented-name': 'Davids (KS)', '@party': 'D', '@state': 'KS', '@role': 'legislator', '#text': 'Davids (KS)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000626', '@sort-field': 'Davidson', '@unaccented-name': 'Davidson', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Davidson'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000096', '@sort-field': 'Davis (IL)', '@unaccented-name': 'Davis (IL)', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Davis (IL)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000230', '@sort-field': 'Davis (NC)', '@unaccented-name': 'Davis (NC)', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Davis (NC)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000594', '@sort-field': 'De La Cruz', '@unaccented-name': 'De La Cruz', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'De La Cruz'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000631', '@sort-field': 'Dean (PA)', '@unaccented-name': 'Dean (PA)', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Dean (PA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000197', '@sort-field': 'DeGette', '@unaccented-name': 'DeGette', '@party': 'D', '@state': 'CO', '@role': 'legislator', '#text': 'DeGette'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000216', '@sort-field': 'DeLauro', '@unaccented-name': 'DeLauro', '@party': 'D', '@state': 'CT', '@role': 'legislator', '#text': 'DeLauro'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000617', '@sort-field': 'DelBene', '@unaccented-name': 'DelBene', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'DelBene'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000530', '@sort-field': 'Deluzio', '@unaccented-name': 'Deluzio', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Deluzio'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000623', '@sort-field': 'DeSaulnier', '@unaccented-name': 'DeSaulnier', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'DeSaulnier'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000616', '@sort-field': 'DesJarlais', '@unaccented-name': 'DesJarlais', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'DesJarlais'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000600', '@sort-field': 'Diaz-Balart', '@unaccented-name': 'Diaz-Balart', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Diaz-Balart'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000624', '@sort-field': 'Dingell', '@unaccented-name': 'Dingell', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Dingell'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000399', '@sort-field': 'Doggett', '@unaccented-name': 'Doggett', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Doggett'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'D000032', '@sort-field': 'Donalds', '@unaccented-name': 'Donalds', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Donalds'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000633', '@sort-field': 'Duarte', '@unaccented-name': 'Duarte', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Duarte'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000615', '@sort-field': 'Duncan', '@unaccented-name': 'Duncan', '@party': 'R', '@state': 'SC', '@role': 'legislator', '#text': 'Duncan'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'D000628', '@sort-field': 'Dunn (FL)', '@unaccented-name': 'Dunn (FL)', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Dunn (FL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'E000246', '@sort-field': 'Edwards', '@unaccented-name': 'Edwards', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'Edwards'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'E000071', '@sort-field': 'Ellzey', '@unaccented-name': 'Ellzey', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Ellzey'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'E000294', '@sort-field': 'Emmer', '@unaccented-name': 'Emmer', '@party': 'R', '@state': 'MN', '@role': 'legislator', '#text': 'Emmer'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'E000299', '@sort-field': 'Escobar', '@unaccented-name': 'Escobar', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Escobar'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'E000215', '@sort-field': 'Eshoo', '@unaccented-name': 'Eshoo', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Eshoo'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'E000297', '@sort-field': 'Espaillat', '@unaccented-name': 'Espaillat', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Espaillat'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'E000298', '@sort-field': 'Estes', '@unaccented-name': 'Estes', '@party': 'R', '@state': 'KS', '@role': 'legislator', '#text': 'Estes'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'E000296', '@sort-field': 'Evans', '@unaccented-name': 'Evans', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Evans'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'E000235', '@sort-field': 'Ezell', '@unaccented-name': 'Ezell', '@party': 'R', '@state': 'MS', '@role': 'legislator', '#text': 'Ezell'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000246', '@sort-field': 'Fallon', '@unaccented-name': 'Fallon', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Fallon'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000446', '@sort-field': 'Feenstra', '@unaccented-name': 'Feenstra', '@party': 'R', '@state': 'IA', '@role': 'legislator', '#text': 'Feenstra'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000465', '@sort-field': 'Ferguson', '@unaccented-name': 'Ferguson', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Ferguson'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000475', '@sort-field': 'Finstad', '@unaccented-name': 'Finstad', '@party': 'R', '@state': 'MN', '@role': 'legislator', '#text': 'Finstad'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000470', '@sort-field': 'Fischbach', '@unaccented-name': 'Fischbach', '@party': 'R', '@state': 'MN', '@role': 'legislator', '#text': 'Fischbach'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000471', '@sort-field': 'Fitzgerald', '@unaccented-name': 'Fitzgerald', '@party': 'R', '@state': 'WI', '@role': 'legislator', '#text': 'Fitzgerald'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000466', '@sort-field': 'Fitzpatrick', '@unaccented-name': 'Fitzpatrick', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Fitzpatrick'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'F000459', '@sort-field': 'Fleischmann', '@unaccented-name': 'Fleischmann', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Fleischmann'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000468', '@sort-field': 'Fletcher', '@unaccented-name': 'Fletcher', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Fletcher'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'F000474', '@sort-field': 'Flood', '@unaccented-name': 'Flood', '@party': 'R', '@state': 'NE', '@role': 'legislator', '#text': 'Flood'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000454', '@sort-field': 'Foster', '@unaccented-name': 'Foster', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Foster'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'F000477', '@sort-field': 'Foushee', '@unaccented-name': 'Foushee', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Foushee'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'F000450', '@sort-field': 'Foxx', '@unaccented-name': 'Foxx', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'Foxx'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000462', '@sort-field': 'Frankel, Lois', '@unaccented-name': 'Frankel, Lois', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Frankel, Lois'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'F000472', '@sort-field': 'Franklin, C. Scott', '@unaccented-name': 'Franklin, C. Scott', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Franklin, C. Scott'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000476', '@sort-field': 'Frost', '@unaccented-name': 'Frost', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Frost'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'F000478', '@sort-field': 'Fry', '@unaccented-name': 'Fry', '@party': 'R', '@state': 'SC', '@role': 'legislator', '#text': 'Fry'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'F000469', '@sort-field': 'Fulcher', '@unaccented-name': 'Fulcher', '@party': 'R', '@state': 'ID', '@role': 'legislator', '#text': 'Fulcher'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000578', '@sort-field': 'Gaetz', '@unaccented-name': 'Gaetz', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Gaetz'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000579', '@sort-field': 'Gallagher', '@unaccented-name': 'Gallagher', '@party': 'R', '@state': 'WI', '@role': 'legislator', '#text': 'Gallagher'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000574', '@sort-field': 'Gallego', '@unaccented-name': 'Gallego', '@party': 'D', '@state': 'AZ', '@role': 'legislator', '#text': 'Gallego'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000559', '@sort-field': 'Garamendi', '@unaccented-name': 'Garamendi', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Garamendi'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000597', '@sort-field': 'Garbarino', '@unaccented-name': 'Garbarino', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Garbarino'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000586', '@sort-field': 'Garcia (IL)', '@unaccented-name': 'Garcia (IL)', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'García (IL)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000587', '@sort-field': 'Garcia (TX)', '@unaccented-name': 'Garcia (TX)', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Garcia (TX)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000061', '@sort-field': 'Garcia, Mike', '@unaccented-name': 'Garcia, Mike', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Garcia, Mike'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000598', '@sort-field': 'Garcia, Robert', '@unaccented-name': 'Garcia, Robert', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Garcia, Robert'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000593', '@sort-field': 'Gimenez', '@unaccented-name': 'Gimenez', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Gimenez'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000592', '@sort-field': 'Golden (ME)', '@unaccented-name': 'Golden (ME)', '@party': 'D', '@state': 'ME', '@role': 'legislator', '#text': 'Golden (ME)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000599', '@sort-field': 'Goldman (NY)', '@unaccented-name': 'Goldman (NY)', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Goldman (NY)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000585', '@sort-field': 'Gomez', '@unaccented-name': 'Gomez', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Gomez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000594', '@sort-field': 'Gonzales, Tony', '@unaccented-name': 'Gonzales, Tony', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Gonzales, Tony'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000581', '@sort-field': 'Gonzalez, Vicente', '@unaccented-name': 'Gonzalez, Vicente', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Gonzalez, Vicente'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000595', '@sort-field': 'Good (VA)', '@unaccented-name': 'Good (VA)', '@party': 'R', '@state': 'VA', '@role': 'legislator', '#text': 'Good (VA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000589', '@sort-field': 'Gooden (TX)', '@unaccented-name': 'Gooden (TX)', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Gooden (TX)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000565', '@sort-field': 'Gosar', '@unaccented-name': 'Gosar', '@party': 'R', '@state': 'AZ', '@role': 'legislator', '#text': 'Gosar'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000583', '@sort-field': 'Gottheimer', '@unaccented-name': 'Gottheimer', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Gottheimer'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000377', '@sort-field': 'Granger', '@unaccented-name': 'Granger', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Granger'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000577', '@sort-field': 'Graves (LA)', '@unaccented-name': 'Graves (LA)', '@party': 'R', '@state': 'LA', '@role': 'legislator', '#text': 'Graves (LA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000546', '@sort-field': 'Graves (MO)', '@unaccented-name': 'Graves (MO)', '@party': 'R', '@state': 'MO', '@role': 'legislator', '#text': 'Graves (MO)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000590', '@sort-field': 'Green (TN)', '@unaccented-name': 'Green (TN)', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Green (TN)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000553', '@sort-field': 'Green, Al (TX)', '@unaccented-name': 'Green, Al (TX)', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Green, Al (TX)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000596', '@sort-field': 'Greene (GA)', '@unaccented-name': 'Greene (GA)', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Greene (GA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000568', '@sort-field': 'Griffith', '@unaccented-name': 'Griffith', '@party': 'R', '@state': 'VA', '@role': 'legislator', '#text': 'Griffith'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000551', '@sort-field': 'Grijalva', '@unaccented-name': 'Grijalva', '@party': 'D', '@state': 'AZ', '@role': 'legislator', '#text': 'Grijalva'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'G000576', '@sort-field': 'Grothman', '@unaccented-name': 'Grothman', '@party': 'R', '@state': 'WI', '@role': 'legislator', '#text': 'Grothman'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000591', '@sort-field': 'Guest', '@unaccented-name': 'Guest', '@party': 'R', '@state': 'MS', '@role': 'legislator', '#text': 'Guest'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000558', '@sort-field': 'Guthrie', '@unaccented-name': 'Guthrie', '@party': 'R', '@state': 'KY', '@role': 'legislator', '#text': 'Guthrie'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001096', '@sort-field': 'Hageman', '@unaccented-name': 'Hageman', '@party': 'R', '@state': 'WY', '@role': 'legislator', '#text': 'Hageman'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001090', '@sort-field': 'Harder (CA)', '@unaccented-name': 'Harder (CA)', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Harder (CA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001052', '@sort-field': 'Harris', '@unaccented-name': 'Harris', '@party': 'R', '@state': 'MD', '@role': 'legislator', '#text': 'Harris'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001086', '@sort-field': 'Harshbarger', '@unaccented-name': 'Harshbarger', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Harshbarger'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001081', '@sort-field': 'Hayes', '@unaccented-name': 'Hayes', '@party': 'D', '@state': 'CT', '@role': 'legislator', '#text': 'Hayes'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001082', '@sort-field': 'Hern', '@unaccented-name': 'Hern', '@party': 'R', '@state': 'OK', '@role': 'legislator', '#text': 'Hern'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001077', '@sort-field': 'Higgins (LA)', '@unaccented-name': 'Higgins (LA)', '@party': 'R', '@state': 'LA', '@role': 'legislator', '#text': 'Higgins (LA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001038', '@sort-field': 'Higgins (NY)', '@unaccented-name': 'Higgins (NY)', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Higgins (NY)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001072', '@sort-field': 'Hill', '@unaccented-name': 'Hill', '@party': 'R', '@state': 'AR', '@role': 'legislator', '#text': 'Hill'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001047', '@sort-field': 'Himes', '@unaccented-name': 'Himes', '@party': 'D', '@state': 'CT', '@role': 'legislator', '#text': 'Himes'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001091', '@sort-field': 'Hinson', '@unaccented-name': 'Hinson', '@party': 'R', '@state': 'IA', '@role': 'legislator', '#text': 'Hinson'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001066', '@sort-field': 'Horsford', '@unaccented-name': 'Horsford', '@party': 'D', '@state': 'NV', '@role': 'legislator', '#text': 'Horsford'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001093', '@sort-field': 'Houchin', '@unaccented-name': 'Houchin', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Houchin'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001085', '@sort-field': 'Houlahan', '@unaccented-name': 'Houlahan', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Houlahan'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H000874', '@sort-field': 'Hoyer', '@unaccented-name': 'Hoyer', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Hoyer'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001094', '@sort-field': 'Hoyle (OR)', '@unaccented-name': 'Hoyle (OR)', '@party': 'D', '@state': 'OR', '@role': 'legislator', '#text': 'Hoyle (OR)'
                    }, 'vote': 'Not Voting'
                },
                {'legislator': {'@name-id': 'H001067', '@sort-field': 'Hudson', '@unaccented-name': 'Hudson', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'Hudson'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001068', '@sort-field': 'Huffman', '@unaccented-name': 'Huffman', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Huffman'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'H001058', '@sort-field': 'Huizenga', '@unaccented-name': 'Huizenga', '@party': 'R', '@state': 'MI', '@role': 'legislator', '#text': 'Huizenga'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'H001095', '@sort-field': 'Hunt', '@unaccented-name': 'Hunt', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Hunt'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'I000056', '@sort-field': 'Issa', '@unaccented-name': 'Issa', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Issa'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'I000058', '@sort-field': 'Ivey', '@unaccented-name': 'Ivey', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Ivey'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000309', '@sort-field': 'Jackson (IL)', '@unaccented-name': 'Jackson (IL)', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Jackson (IL)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000308', '@sort-field': 'Jackson (NC)', '@unaccented-name': 'Jackson (NC)', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Jackson (NC)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000304', '@sort-field': 'Jackson (TX)', '@unaccented-name': 'Jackson (TX)', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Jackson (TX)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000032', '@sort-field': 'Jackson Lee', '@unaccented-name': 'Jackson Lee', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Jackson Lee'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000305', '@sort-field': 'Jacobs', '@unaccented-name': 'Jacobs', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Jacobs'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000307', '@sort-field': 'James', '@unaccented-name': 'James', '@party': 'R', '@state': 'MI', '@role': 'legislator', '#text': 'James'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000298', '@sort-field': 'Jayapal', '@unaccented-name': 'Jayapal', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Jayapal'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000294', '@sort-field': 'Jeffries', '@unaccented-name': 'Jeffries', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Jeffries'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000288', '@sort-field': 'Johnson (GA)', '@unaccented-name': 'Johnson (GA)', '@party': 'D', '@state': 'GA', '@role': 'legislator', '#text': 'Johnson (GA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'J000299', '@sort-field': 'Johnson (LA)', '@unaccented-name': 'Johnson (LA)', '@party': 'R', '@state': 'LA', '@role': 'legislator', '#text': 'Johnson (LA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000292', '@sort-field': 'Johnson (OH)', '@unaccented-name': 'Johnson (OH)', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Johnson (OH)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000301', '@sort-field': 'Johnson (SD)', '@unaccented-name': 'Johnson (SD)', '@party': 'R', '@state': 'SD', '@role': 'legislator', '#text': 'Johnson (SD)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000289', '@sort-field': 'Jordan', '@unaccented-name': 'Jordan', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Jordan'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000295', '@sort-field': 'Joyce (OH)', '@unaccented-name': 'Joyce (OH)', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Joyce (OH)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'J000302', '@sort-field': 'Joyce (PA)', '@unaccented-name': 'Joyce (PA)', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Joyce (PA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000400', '@sort-field': 'Kamlager-Dove', '@unaccented-name': 'Kamlager-Dove', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Kamlager-Dove'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000009', '@sort-field': 'Kaptur', '@unaccented-name': 'Kaptur', '@party': 'D', '@state': 'OH', '@role': 'legislator', '#text': 'Kaptur'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000398', '@sort-field': 'Kean (NJ)', '@unaccented-name': 'Kean (NJ)', '@party': 'R', '@state': 'NJ', '@role': 'legislator', '#text': 'Kean (NJ)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000375', '@sort-field': 'Keating', '@unaccented-name': 'Keating', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Keating'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000385', '@sort-field': 'Kelly (IL)', '@unaccented-name': 'Kelly (IL)', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Kelly (IL)'
                    }, 'vote': 'Not Voting'
                },
                {'legislator': {'@name-id': 'K000388', '@sort-field': 'Kelly (MS)', '@unaccented-name': 'Kelly (MS)', '@party': 'R', '@state': 'MS', '@role': 'legislator', '#text': 'Kelly (MS)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000376', '@sort-field': 'Kelly (PA)', '@unaccented-name': 'Kelly (PA)', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Kelly (PA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000389', '@sort-field': 'Khanna', '@unaccented-name': 'Khanna', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Khanna'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000399', '@sort-field': 'Kiggans (VA)', '@unaccented-name': 'Kiggans (VA)', '@party': 'R', '@state': 'VA', '@role': 'legislator', '#text': 'Kiggans (VA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000380', '@sort-field': 'Kildee', '@unaccented-name': 'Kildee', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Kildee'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000401', '@sort-field': 'Kiley', '@unaccented-name': 'Kiley', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Kiley'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000381', '@sort-field': 'Kilmer', '@unaccented-name': 'Kilmer', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Kilmer'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000397', '@sort-field': 'Kim (CA)', '@unaccented-name': 'Kim (CA)', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Kim (CA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'K000394', '@sort-field': 'Kim (NJ)', '@unaccented-name': 'Kim (NJ)', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Kim (NJ)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000391', '@sort-field': 'Krishnamoorthi', '@unaccented-name': 'Krishnamoorthi', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Krishnamoorthi'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000382', '@sort-field': 'Kuster', '@unaccented-name': 'Kuster', '@party': 'D', '@state': 'NH', '@role': 'legislator', '#text': 'Kuster'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'K000392', '@sort-field': 'Kustoff', '@unaccented-name': 'Kustoff', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Kustoff'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000585', '@sort-field': 'LaHood', '@unaccented-name': 'LaHood', '@party': 'R', '@state': 'IL', '@role': 'legislator', '#text': 'LaHood'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000598', '@sort-field': 'LaLota', '@unaccented-name': 'LaLota', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'LaLota'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000578', '@sort-field': 'LaMalfa', '@unaccented-name': 'LaMalfa', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'LaMalfa'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000564', '@sort-field': 'Lamborn', '@unaccented-name': 'Lamborn', '@party': 'R', '@state': 'CO', '@role': 'legislator', '#text': 'Lamborn'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000601', '@sort-field': 'Landsman', '@unaccented-name': 'Landsman', '@party': 'D', '@state': 'OH', '@role': 'legislator', '#text': 'Landsman'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000600', '@sort-field': 'Langworthy', '@unaccented-name': 'Langworthy', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Langworthy'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000560', '@sort-field': 'Larsen (WA)', '@unaccented-name': 'Larsen (WA)', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Larsen (WA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000557', '@sort-field': 'Larson (CT)', '@unaccented-name': 'Larson (CT)', '@party': 'D', '@state': 'CT', '@role': 'legislator', '#text': 'Larson (CT)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000566', '@sort-field': 'Latta', '@unaccented-name': 'Latta', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Latta'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000266', '@sort-field': 'LaTurner', '@unaccented-name': 'LaTurner', '@party': 'R', '@state': 'KS', '@role': 'legislator', '#text': 'LaTurner'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000599', '@sort-field': 'Lawler', '@unaccented-name': 'Lawler', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Lawler'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000551', '@sort-field': 'Lee (CA)', '@unaccented-name': 'Lee (CA)', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Lee (CA)'
                    }, 'vote': 'Not Voting'
                },
                {'legislator': {'@name-id': 'L000597', '@sort-field': 'Lee (FL)', '@unaccented-name': 'Lee (FL)', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Lee (FL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000590', '@sort-field': 'Lee (NV)', '@unaccented-name': 'Lee (NV)', '@party': 'D', '@state': 'NV', '@role': 'legislator', '#text': 'Lee (NV)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000602', '@sort-field': 'Lee (PA)', '@unaccented-name': 'Lee (PA)', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Lee (PA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000273', '@sort-field': 'Leger Fernandez', '@unaccented-name': 'Leger Fernandez', '@party': 'D', '@state': 'NM', '@role': 'legislator', '#text': 'Leger Fernandez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000589', '@sort-field': 'Lesko', '@unaccented-name': 'Lesko', '@party': 'R', '@state': 'AZ', '@role': 'legislator', '#text': 'Lesko'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000595', '@sort-field': 'Letlow', '@unaccented-name': 'Letlow', '@party': 'R', '@state': 'LA', '@role': 'legislator', '#text': 'Letlow'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000593', '@sort-field': 'Levin', '@unaccented-name': 'Levin', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Levin'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000582', '@sort-field': 'Lieu', '@unaccented-name': 'Lieu', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Lieu'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000397', '@sort-field': 'Lofgren', '@unaccented-name': 'Lofgren', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Lofgren'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'L000583', '@sort-field': 'Loudermilk', '@unaccented-name': 'Loudermilk', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Loudermilk'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000491', '@sort-field': 'Lucas', '@unaccented-name': 'Lucas', '@party': 'R', '@state': 'OK', '@role': 'legislator', '#text': 'Lucas'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000569', '@sort-field': 'Luetkemeyer', '@unaccented-name': 'Luetkemeyer', '@party': 'R', '@state': 'MO', '@role': 'legislator', '#text': 'Luetkemeyer'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000596', '@sort-field': 'Luna', '@unaccented-name': 'Luna', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Luna'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000603', '@sort-field': 'Luttrell', '@unaccented-name': 'Luttrell', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Luttrell'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'L000562', '@sort-field': 'Lynch', '@unaccented-name': 'Lynch', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Lynch'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M000194', '@sort-field': 'Mace', '@unaccented-name': 'Mace', '@party': 'R', '@state': 'SC', '@role': 'legislator', '#text': 'Mace'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001223', '@sort-field': 'Magaziner', '@unaccented-name': 'Magaziner', '@party': 'D', '@state': 'RI', '@role': 'legislator', '#text': 'Magaziner'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M000317', '@sort-field': 'Malliotakis', '@unaccented-name': 'Malliotakis', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Malliotakis'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M000871', '@sort-field': 'Mann', '@unaccented-name': 'Mann', '@party': 'R', '@state': 'KS', '@role': 'legislator', '#text': 'Mann'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001135', '@sort-field': 'Manning', '@unaccented-name': 'Manning', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Manning'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001184', '@sort-field': 'Massie', '@unaccented-name': 'Massie', '@party': 'R', '@state': 'KY', '@role': 'legislator', '#text': 'Massie'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001199', '@sort-field': 'Mast', '@unaccented-name': 'Mast', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Mast'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001163', '@sort-field': 'Matsui', '@unaccented-name': 'Matsui', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Matsui'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001208', '@sort-field': 'McBath', '@unaccented-name': 'McBath', '@party': 'D', '@state': 'GA', '@role': 'legislator', '#text': 'McBath'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001165', '@sort-field': 'McCarthy', '@unaccented-name': 'McCarthy', '@party': 'R', '@state': 'CA', '@role': 'speaker', '#text': 'McCarthy'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001157', '@sort-field': 'McCaul', '@unaccented-name': 'McCaul', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'McCaul'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001136', '@sort-field': 'McClain', '@unaccented-name': 'McClain', '@party': 'R', '@state': 'MI', '@role': 'legislator', '#text': 'McClain'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001227', '@sort-field': 'McClellan', '@unaccented-name': 'McClellan', '@party': 'D', '@state': 'VA', '@role': 'legislator', '#text': 'McClellan'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001177', '@sort-field': 'McClintock', '@unaccented-name': 'McClintock', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'McClintock'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001143', '@sort-field': 'McCollum', '@unaccented-name': 'McCollum', '@party': 'D', '@state': 'MN', '@role': 'legislator', '#text': 'McCollum'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001218', '@sort-field': 'McCormick', '@unaccented-name': 'McCormick', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'McCormick'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001220', '@sort-field': 'McGarvey', '@unaccented-name': 'McGarvey', '@party': 'D', '@state': 'KY', '@role': 'legislator', '#text': 'McGarvey'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M000312', '@sort-field': 'McGovern', '@unaccented-name': 'McGovern', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'McGovern'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001156', '@sort-field': 'McHenry', '@unaccented-name': 'McHenry', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'McHenry'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001137', '@sort-field': 'Meeks', '@unaccented-name': 'Meeks', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Meeks'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001226', '@sort-field': 'Menendez', '@unaccented-name': 'Menendez', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Menendez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001188', '@sort-field': 'Meng', '@unaccented-name': 'Meng', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Meng'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001204', '@sort-field': 'Meuser', '@unaccented-name': 'Meuser', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Meuser'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M000687', '@sort-field': 'Mfume', '@unaccented-name': 'Mfume', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Mfume'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001211', '@sort-field': 'Miller (IL)', '@unaccented-name': 'Miller (IL)', '@party': 'R', '@state': 'IL', '@role': 'legislator', '#text': 'Miller (IL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001222', '@sort-field': 'Miller (OH)', '@unaccented-name': 'Miller (OH)', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Miller (OH)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001205', '@sort-field': 'Miller (WV)', '@unaccented-name': 'Miller (WV)', '@party': 'R', '@state': 'WV', '@role': 'legislator', '#text': 'Miller (WV)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001215', '@sort-field': 'Miller-Meeks', '@unaccented-name': 'Miller-Meeks', '@party': 'R', '@state': 'IA', '@role': 'legislator', '#text': 'Miller-Meeks'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001216', '@sort-field': 'Mills', '@unaccented-name': 'Mills', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Mills'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001221', '@sort-field': 'Molinaro', '@unaccented-name': 'Molinaro', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Molinaro'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001194', '@sort-field': 'Moolenaar', '@unaccented-name': 'Moolenaar', '@party': 'R', '@state': 'MI', '@role': 'legislator', '#text': 'Moolenaar'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001195', '@sort-field': 'Mooney', '@unaccented-name': 'Mooney', '@party': 'R', '@state': 'WV', '@role': 'legislator', '#text': 'Mooney'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001212', '@sort-field': 'Moore (AL)', '@unaccented-name': 'Moore (AL)', '@party': 'R', '@state': 'AL', '@role': 'legislator', '#text': 'Moore (AL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001213', '@sort-field': 'Moore (UT)', '@unaccented-name': 'Moore (UT)', '@party': 'R', '@state': 'UT', '@role': 'legislator', '#text': 'Moore (UT)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001160', '@sort-field': 'Moore (WI)', '@unaccented-name': 'Moore (WI)', '@party': 'D', '@state': 'WI', '@role': 'legislator', '#text': 'Moore (WI)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001224', '@sort-field': 'Moran', '@unaccented-name': 'Moran', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Moran'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001206', '@sort-field': 'Morelle', '@unaccented-name': 'Morelle', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Morelle'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001217', '@sort-field': 'Moskowitz', '@unaccented-name': 'Moskowitz', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Moskowitz'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001196', '@sort-field': 'Moulton', '@unaccented-name': 'Moulton', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Moulton'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001214', '@sort-field': 'Mrvan', '@unaccented-name': 'Mrvan', '@party': 'D', '@state': 'IN', '@role': 'legislator', '#text': 'Mrvan'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001225', '@sort-field': 'Mullin', '@unaccented-name': 'Mullin', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Mullin'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'M001210', '@sort-field': 'Murphy', '@unaccented-name': 'Murphy', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'Murphy'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'N000002', '@sort-field': 'Nadler', '@unaccented-name': 'Nadler', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Nadler'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'N000179', '@sort-field': 'Napolitano', '@unaccented-name': 'Napolitano', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Napolitano'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'N000015', '@sort-field': 'Neal', '@unaccented-name': 'Neal', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Neal'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'N000191', '@sort-field': 'Neguse', '@unaccented-name': 'Neguse', '@party': 'D', '@state': 'CO', '@role': 'legislator', '#text': 'Neguse'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'N000026', '@sort-field': 'Nehls', '@unaccented-name': 'Nehls', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Nehls'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'N000189', '@sort-field': 'Newhouse', '@unaccented-name': 'Newhouse', '@party': 'R', '@state': 'WA', '@role': 'legislator', '#text': 'Newhouse'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'N000194', '@sort-field': 'Nickel', '@unaccented-name': 'Nickel', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Nickel'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'N000188', '@sort-field': 'Norcross', '@unaccented-name': 'Norcross', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Norcross'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'N000190', '@sort-field': 'Norman', '@unaccented-name': 'Norman', '@party': 'R', '@state': 'SC', '@role': 'legislator', '#text': 'Norman'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'N000193', '@sort-field': 'Nunn (IA)', '@unaccented-name': 'Nunn (IA)', '@party': 'R', '@state': 'IA', '@role': 'legislator', '#text': 'Nunn (IA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'O000019', '@sort-field': 'Obernolte', '@unaccented-name': 'Obernolte', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Obernolte'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'O000172', '@sort-field': 'Ocasio-Cortez', '@unaccented-name': 'Ocasio-Cortez', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Ocasio-Cortez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'O000175', '@sort-field': 'Ogles', '@unaccented-name': 'Ogles', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Ogles'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'O000173', '@sort-field': 'Omar', '@unaccented-name': 'Omar', '@party': 'D', '@state': 'MN', '@role': 'legislator', '#text': 'Omar'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'O000086', '@sort-field': 'Owens', '@unaccented-name': 'Owens', '@party': 'R', '@state': 'UT', '@role': 'legislator', '#text': 'Owens'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'P000034', '@sort-field': 'Pallone', '@unaccented-name': 'Pallone', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Pallone'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000609', '@sort-field': 'Palmer', '@unaccented-name': 'Palmer', '@party': 'R', '@state': 'AL', '@role': 'legislator', '#text': 'Palmer'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'P000613', '@sort-field': 'Panetta', '@unaccented-name': 'Panetta', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Panetta'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000614', '@sort-field': 'Pappas', '@unaccented-name': 'Pappas', '@party': 'D', '@state': 'NH', '@role': 'legislator', '#text': 'Pappas'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000096', '@sort-field': 'Pascrell', '@unaccented-name': 'Pascrell', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Pascrell'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000604', '@sort-field': 'Payne', '@unaccented-name': 'Payne', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Payne'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000197', '@sort-field': 'Pelosi', '@unaccented-name': 'Pelosi', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Pelosi'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000619', '@sort-field': 'Peltola', '@unaccented-name': 'Peltola', '@party': 'D', '@state': 'AK', '@role': 'legislator', '#text': 'Peltola'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000615', '@sort-field': 'Pence', '@unaccented-name': 'Pence', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Pence'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'G000600', '@sort-field': 'Perez', '@unaccented-name': 'Perez', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Perez'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'P000605', '@sort-field': 'Perry', '@unaccented-name': 'Perry', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Perry'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'P000608', '@sort-field': 'Peters', '@unaccented-name': 'Peters', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Peters'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000620', '@sort-field': 'Pettersen', '@unaccented-name': 'Pettersen', '@party': 'D', '@state': 'CO', '@role': 'legislator', '#text': 'Pettersen'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000048', '@sort-field': 'Pfluger', '@unaccented-name': 'Pfluger', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Pfluger'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'P000616', '@sort-field': 'Phillips', '@unaccented-name': 'Phillips', '@party': 'D', '@state': 'MN', '@role': 'legislator', '#text': 'Phillips'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000597', '@sort-field': 'Pingree', '@unaccented-name': 'Pingree', '@party': 'D', '@state': 'ME', '@role': 'legislator', '#text': 'Pingree'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000607', '@sort-field': 'Pocan', '@unaccented-name': 'Pocan', '@party': 'D', '@state': 'WI', '@role': 'legislator', '#text': 'Pocan'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000618', '@sort-field': 'Porter', '@unaccented-name': 'Porter', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Porter'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'P000599', '@sort-field': 'Posey', '@unaccented-name': 'Posey', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Posey'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'P000617', '@sort-field': 'Pressley', '@unaccented-name': 'Pressley', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Pressley'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'Q000023', '@sort-field': 'Quigley', '@unaccented-name': 'Quigley', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Quigley'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'R000617', '@sort-field': 'Ramirez', '@unaccented-name': 'Ramirez', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Ramirez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'R000606', '@sort-field': 'Raskin', '@unaccented-name': 'Raskin', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Raskin'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'R000610', '@sort-field': 'Reschenthaler', '@unaccented-name': 'Reschenthaler', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Reschenthaler'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'M001159', '@sort-field': 'Rodgers (WA)', '@unaccented-name': 'Rodgers (WA)', '@party': 'R', '@state': 'WA', '@role': 'legislator', '#text': 'Rodgers (WA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000575', '@sort-field': 'Rogers (AL)', '@unaccented-name': 'Rogers (AL)', '@party': 'R', '@state': 'AL', '@role': 'legislator', '#text': 'Rogers (AL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000395', '@sort-field': 'Rogers (KY)', '@unaccented-name': 'Rogers (KY)', '@party': 'R', '@state': 'KY', '@role': 'legislator', '#text': 'Rogers (KY)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000612', '@sort-field': 'Rose', '@unaccented-name': 'Rose', '@party': 'R', '@state': 'TN', '@role': 'legislator', '#text': 'Rose'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000103', '@sort-field': 'Rosendale', '@unaccented-name': 'Rosendale', '@party': 'R', '@state': 'MT', '@role': 'legislator', '#text': 'Rosendale'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000305', '@sort-field': 'Ross', '@unaccented-name': 'Ross', '@party': 'D', '@state': 'NC', '@role': 'legislator', '#text': 'Ross'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'R000603', '@sort-field': 'Rouzer', '@unaccented-name': 'Rouzer', '@party': 'R', '@state': 'NC', '@role': 'legislator', '#text': 'Rouzer'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000614', '@sort-field': 'Roy', '@unaccented-name': 'Roy', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Roy'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000599', '@sort-field': 'Ruiz', '@unaccented-name': 'Ruiz', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Ruiz'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'R000576', '@sort-field': 'Ruppersberger', '@unaccented-name': 'Ruppersberger', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Ruppersberger'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'R000609', '@sort-field': 'Rutherford', '@unaccented-name': 'Rutherford', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Rutherford'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'R000579', '@sort-field': 'Ryan', '@unaccented-name': 'Ryan', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Ryan'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S000168', '@sort-field': 'Salazar', '@unaccented-name': 'Salazar', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Salazar'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001226', '@sort-field': 'Salinas', '@unaccented-name': 'Salinas', '@party': 'D', '@state': 'OR', '@role': 'legislator', '#text': 'Salinas'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001156', '@sort-field': 'Sanchez', '@unaccented-name': 'Sanchez', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Sánchez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001222', '@sort-field': 'Santos', '@unaccented-name': 'Santos', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Santos'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001168', '@sort-field': 'Sarbanes', '@unaccented-name': 'Sarbanes', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Sarbanes'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001176', '@sort-field': 'Scalise', '@unaccented-name': 'Scalise', '@party': 'R', '@state': 'LA', '@role': 'legislator', '#text': 'Scalise'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001205', '@sort-field': 'Scanlon', '@unaccented-name': 'Scanlon', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Scanlon'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001145', '@sort-field': 'Schakowsky', '@unaccented-name': 'Schakowsky', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Schakowsky'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001150', '@sort-field': 'Schiff', '@unaccented-name': 'Schiff', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Schiff'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001190', '@sort-field': 'Schneider', '@unaccented-name': 'Schneider', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Schneider'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001221', '@sort-field': 'Scholten', '@unaccented-name': 'Scholten', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Scholten'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001216', '@sort-field': 'Schrier', '@unaccented-name': 'Schrier', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Schrier'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001183', '@sort-field': 'Schweikert', '@unaccented-name': 'Schweikert', '@party': 'R', '@state': 'AZ', '@role': 'legislator', '#text': 'Schweikert'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S000185', '@sort-field': 'Scott (VA)', '@unaccented-name': 'Scott (VA)', '@party': 'D', '@state': 'VA', '@role': 'legislator', '#text': 'Scott (VA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001189', '@sort-field': 'Scott, Austin', '@unaccented-name': 'Scott, Austin', '@party': 'R', '@state': 'GA', '@role': 'legislator', '#text': 'Scott, Austin'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001157', '@sort-field': 'Scott, David', '@unaccented-name': 'Scott, David', '@party': 'D', '@state': 'GA', '@role': 'legislator', '#text': 'Scott, David'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001224', '@sort-field': 'Self', '@unaccented-name': 'Self', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Self'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S000250', '@sort-field': 'Sessions', '@unaccented-name': 'Sessions', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Sessions'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001185', '@sort-field': 'Sewell', '@unaccented-name': 'Sewell', '@party': 'D', '@state': 'AL', '@role': 'legislator', '#text': 'Sewell'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S000344', '@sort-field': 'Sherman', '@unaccented-name': 'Sherman', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Sherman'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001207', '@sort-field': 'Sherrill', '@unaccented-name': 'Sherrill', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Sherrill'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001148', '@sort-field': 'Simpson', '@unaccented-name': 'Simpson', '@party': 'R', '@state': 'ID', '@role': 'legislator', '#text': 'Simpson'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001208', '@sort-field': 'Slotkin', '@unaccented-name': 'Slotkin', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Slotkin'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001195', '@sort-field': 'Smith (MO)', '@unaccented-name': 'Smith (MO)', '@party': 'R', '@state': 'MO', '@role': 'legislator', '#text': 'Smith (MO)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001172', '@sort-field': 'Smith (NE)', '@unaccented-name': 'Smith (NE)', '@party': 'R', '@state': 'NE', '@role': 'legislator', '#text': 'Smith (NE)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S000522', '@sort-field': 'Smith (NJ)', '@unaccented-name': 'Smith (NJ)', '@party': 'R', '@state': 'NJ', '@role': 'legislator', '#text': 'Smith (NJ)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S000510', '@sort-field': 'Smith (WA)', '@unaccented-name': 'Smith (WA)', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Smith (WA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001199', '@sort-field': 'Smucker', '@unaccented-name': 'Smucker', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Smucker'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001225', '@sort-field': 'Sorensen', '@unaccented-name': 'Sorensen', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Sorensen'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001200', '@sort-field': 'Soto', '@unaccented-name': 'Soto', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Soto'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001209', '@sort-field': 'Spanberger', '@unaccented-name': 'Spanberger', '@party': 'D', '@state': 'VA', '@role': 'legislator', '#text': 'Spanberger'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S000929', '@sort-field': 'Spartz', '@unaccented-name': 'Spartz', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Spartz'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001218', '@sort-field': 'Stansbury', '@unaccented-name': 'Stansbury', '@party': 'D', '@state': 'NM', '@role': 'legislator', '#text': 'Stansbury'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001211', '@sort-field': 'Stanton', '@unaccented-name': 'Stanton', '@party': 'D', '@state': 'AZ', '@role': 'legislator', '#text': 'Stanton'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001212', '@sort-field': 'Stauber', '@unaccented-name': 'Stauber', '@party': 'R', '@state': 'MN', '@role': 'legislator', '#text': 'Stauber'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001135', '@sort-field': 'Steel', '@unaccented-name': 'Steel', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Steel'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001196', '@sort-field': 'Stefanik', '@unaccented-name': 'Stefanik', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Stefanik'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001213', '@sort-field': 'Steil', '@unaccented-name': 'Steil', '@party': 'R', '@state': 'WI', '@role': 'legislator', '#text': 'Steil'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001214', '@sort-field': 'Steube', '@unaccented-name': 'Steube', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Steube'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001215', '@sort-field': 'Stevens', '@unaccented-name': 'Stevens', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Stevens'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001192', '@sort-field': 'Stewart', '@unaccented-name': 'Stewart', '@party': 'R', '@state': 'UT', '@role': 'legislator', '#text': 'Stewart'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001159', '@sort-field': 'Strickland', '@unaccented-name': 'Strickland', '@party': 'D', '@state': 'WA', '@role': 'legislator', '#text': 'Strickland'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001220', '@sort-field': 'Strong', '@unaccented-name': 'Strong', '@party': 'R', '@state': 'AL', '@role': 'legislator', '#text': 'Strong'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'S001193', '@sort-field': 'Swalwell', '@unaccented-name': 'Swalwell', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Swalwell'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'S001223', '@sort-field': 'Sykes', '@unaccented-name': 'Sykes', '@party': 'D', '@state': 'OH', '@role': 'legislator', '#text': 'Sykes'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000472', '@sort-field': 'Takano', '@unaccented-name': 'Takano', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Takano'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000478', '@sort-field': 'Tenney', '@unaccented-name': 'Tenney', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Tenney'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'T000488', '@sort-field': 'Thanedar', '@unaccented-name': 'Thanedar', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Thanedar'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000460', '@sort-field': 'Thompson (CA)', '@unaccented-name': 'Thompson (CA)', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Thompson (CA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000193', '@sort-field': 'Thompson (MS)', '@unaccented-name': 'Thompson (MS)', '@party': 'D', '@state': 'MS', '@role': 'legislator', '#text': 'Thompson (MS)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000467', '@sort-field': 'Thompson (PA)', '@unaccented-name': 'Thompson (PA)', '@party': 'R', '@state': 'PA', '@role': 'legislator', '#text': 'Thompson (PA)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'T000165', '@sort-field': 'Tiffany', '@unaccented-name': 'Tiffany', '@party': 'R', '@state': 'WI', '@role': 'legislator', '#text': 'Tiffany'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'T000480', '@sort-field': 'Timmons', '@unaccented-name': 'Timmons', '@party': 'R', '@state': 'SC', '@role': 'legislator', '#text': 'Timmons'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'T000468', '@sort-field': 'Titus', '@unaccented-name': 'Titus', '@party': 'D', '@state': 'NV', '@role': 'legislator', '#text': 'Titus'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000481', '@sort-field': 'Tlaib', '@unaccented-name': 'Tlaib', '@party': 'D', '@state': 'MI', '@role': 'legislator', '#text': 'Tlaib'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000487', '@sort-field': 'Tokuda', '@unaccented-name': 'Tokuda', '@party': 'D', '@state': 'HI', '@role': 'legislator', '#text': 'Tokuda'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000469', '@sort-field': 'Tonko', '@unaccented-name': 'Tonko', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Tonko'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000474', '@sort-field': 'Torres (CA)', '@unaccented-name': 'Torres (CA)', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Torres (CA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000486', '@sort-field': 'Torres (NY)', '@unaccented-name': 'Torres (NY)', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Torres (NY)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000482', '@sort-field': 'Trahan', '@unaccented-name': 'Trahan', '@party': 'D', '@state': 'MA', '@role': 'legislator', '#text': 'Trahan'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000483', '@sort-field': 'Trone', '@unaccented-name': 'Trone', '@party': 'D', '@state': 'MD', '@role': 'legislator', '#text': 'Trone'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'T000463', '@sort-field': 'Turner', '@unaccented-name': 'Turner', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Turner'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'U000040', '@sort-field': 'Underwood', '@unaccented-name': 'Underwood', '@party': 'D', '@state': 'IL', '@role': 'legislator', '#text': 'Underwood'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'V000129', '@sort-field': 'Valadao', '@unaccented-name': 'Valadao', '@party': 'R', '@state': 'CA', '@role': 'legislator', '#text': 'Valadao'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'V000133', '@sort-field': 'Van Drew', '@unaccented-name': 'Van Drew', '@party': 'R', '@state': 'NJ', '@role': 'legislator', '#text': 'Van Drew'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'V000134', '@sort-field': 'Van Duyne', '@unaccented-name': 'Van Duyne', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Van Duyne'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'V000135', '@sort-field': 'Van Orden', '@unaccented-name': 'Van Orden', '@party': 'R', '@state': 'WI', '@role': 'legislator', '#text': 'Van Orden'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'V000130', '@sort-field': 'Vargas', '@unaccented-name': 'Vargas', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Vargas'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'V000136', '@sort-field': 'Vasquez', '@unaccented-name': 'Vasquez', '@party': 'D', '@state': 'NM', '@role': 'legislator', '#text': 'Vasquez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'V000131', '@sort-field': 'Veasey', '@unaccented-name': 'Veasey', '@party': 'D', '@state': 'TX', '@role': 'legislator', '#text': 'Veasey'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'V000081', '@sort-field': 'Velazquez', '@unaccented-name': 'Velazquez', '@party': 'D', '@state': 'NY', '@role': 'legislator', '#text': 'Velázquez'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000812', '@sort-field': 'Wagner', '@unaccented-name': 'Wagner', '@party': 'R', '@state': 'MO', '@role': 'legislator', '#text': 'Wagner'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000798', '@sort-field': 'Walberg', '@unaccented-name': 'Walberg', '@party': 'R', '@state': 'MI', '@role': 'legislator', '#text': 'Walberg'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000823', '@sort-field': 'Waltz', '@unaccented-name': 'Waltz', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Waltz'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000797', '@sort-field': 'Wasserman Schultz', '@unaccented-name': 'Wasserman Schultz', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Wasserman Schultz'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000187', '@sort-field': 'Waters', '@unaccented-name': 'Waters', '@party': 'D', '@state': 'CA', '@role': 'legislator', '#text': 'Waters'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000822', '@sort-field': 'Watson Coleman', '@unaccented-name': 'Watson Coleman', '@party': 'D', '@state': 'NJ', '@role': 'legislator', '#text': 'Watson Coleman'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000814', '@sort-field': 'Weber (TX)', '@unaccented-name': 'Weber (TX)', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Weber (TX)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000806', '@sort-field': 'Webster (FL)', '@unaccented-name': 'Webster (FL)', '@party': 'R', '@state': 'FL', '@role': 'legislator', '#text': 'Webster (FL)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000815', '@sort-field': 'Wenstrup', '@unaccented-name': 'Wenstrup', '@party': 'R', '@state': 'OH', '@role': 'legislator', '#text': 'Wenstrup'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000821', '@sort-field': 'Westerman', '@unaccented-name': 'Westerman', '@party': 'R', '@state': 'AR', '@role': 'legislator', '#text': 'Westerman'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000825', '@sort-field': 'Wexton', '@unaccented-name': 'Wexton', '@party': 'D', '@state': 'VA', '@role': 'legislator', '#text': 'Wexton'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000826', '@sort-field': 'Wild', '@unaccented-name': 'Wild', '@party': 'D', '@state': 'PA', '@role': 'legislator', '#text': 'Wild'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000788', '@sort-field': 'Williams (GA)', '@unaccented-name': 'Williams (GA)', '@party': 'D', '@state': 'GA', '@role': 'legislator', '#text': 'Williams (GA)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000828', '@sort-field': 'Williams (NY)', '@unaccented-name': 'Williams (NY)', '@party': 'R', '@state': 'NY', '@role': 'legislator', '#text': 'Williams (NY)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000816', '@sort-field': 'Williams (TX)', '@unaccented-name': 'Williams (TX)', '@party': 'R', '@state': 'TX', '@role': 'legislator', '#text': 'Williams (TX)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000808', '@sort-field': 'Wilson (FL)', '@unaccented-name': 'Wilson (FL)', '@party': 'D', '@state': 'FL', '@role': 'legislator', '#text': 'Wilson (FL)'
                    }, 'vote': 'Nay'
                },
                {'legislator': {'@name-id': 'W000795', '@sort-field': 'Wilson (SC)', '@unaccented-name': 'Wilson (SC)', '@party': 'R', '@state': 'SC', '@role': 'legislator', '#text': 'Wilson (SC)'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000804', '@sort-field': 'Wittman', '@unaccented-name': 'Wittman', '@party': 'R', '@state': 'VA', '@role': 'legislator', '#text': 'Wittman'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'W000809', '@sort-field': 'Womack', '@unaccented-name': 'Womack', '@party': 'R', '@state': 'AR', '@role': 'legislator', '#text': 'Womack'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'Y000067', '@sort-field': 'Yakym', '@unaccented-name': 'Yakym', '@party': 'R', '@state': 'IN', '@role': 'legislator', '#text': 'Yakym'
                    }, 'vote': 'Yea'
                },
                {'legislator': {'@name-id': 'Z000018', '@sort-field': 'Zinke', '@unaccented-name': 'Zinke', '@party': 'R', '@state': 'MT', '@role': 'legislator', '#text': 'Zinke'
                    }, 'vote': 'Yea'
                }
            ]
        }
    }
}